In [394]:
import pandas as pd
import xlwings as xw

import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
%matplotlib inline

rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

시군구 : 법정동코드로 대체(시,구,동으로 나눠 외부 데이터를 적용시에 활용)

번지,본번,부번 : 사용 안함(아파트의 위도, 경도 추출에만 사용함)

단지명 : id로서 활용

도로명 : 번지,본번,부번과 함께 아파트의 위도, 경도 추출시에만 사용

In [395]:
train = pd.read_csv('data/ProcessedData/train_final.csv', encoding='cp949')
train

,시군구,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,위도,경도
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200606,1,"60,000",4,1988,언주로 103,37.481344,127.052912
1,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,20,"72,250",5,1988,언주로 103,37.481344,127.052912
2,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,30,"73,500",8,1988,언주로 103,37.481344,127.052912
3,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200612,9,"76,500",5,1988,언주로 103,37.481344,127.052912
4,서울특별시 강남구 개포동,658,1,개포6차우성아파트1동~8동,79.97,200605,9,"88,000",3,1987,언주로 3,37.476792,127.055926
...,...,...,...,...,...,...,...,...,...,...,...,...,...
825533,서울특별시 중랑구 중화동,450,0,한신2,59.76,201704,26,"33,000",22,1998,동일로 752,37.597009,127.082071
825534,서울특별시 중랑구 중화동,450,0,한신2,84.03,201705,1,"40,200",22,1998,동일로 752,37.597009,127.082071
825535,서울특별시 중랑구 중화동,274,51,한영(101),84.69,201605,17,"30,000",4,2003,동일로144길 74,37.606521,127.082165
825536,서울특별시 중랑구 중화동,274,76,한영(103),67.44,201610,25,"25,700",5,2003,동일로144길 74,37.606521,127.082165


In [396]:
# 거래금액열에 값들의 ','를 없애기

idx = train.index
for i in range(len(train)):
    train.at[idx[i], '거래금액(만원)'] = train['거래금액(만원)'][i].replace(',','')
train['거래금액(만원)'] = train['거래금액(만원)'].astype(int)
train.head()

,시군구,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,위도,경도
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200606,1,60000,4,1988,언주로 103,37.481344,127.052912
1,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,20,72250,5,1988,언주로 103,37.481344,127.052912
2,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,30,73500,8,1988,언주로 103,37.481344,127.052912
3,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200612,9,76500,5,1988,언주로 103,37.481344,127.052912
4,서울특별시 강남구 개포동,658,1,개포6차우성아파트1동~8동,79.97,200605,9,88000,3,1987,언주로 3,37.476792,127.055926


In [397]:
# train 의 시군구 컬럼에서 구이름과 동이름만 가져오기

train['city'], train['gu'], train['dong'] = train['시군구'].str.split(' ',2).str
train.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  This is separate from the ipykernel package so we can avoid doing imports until


,시군구,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,위도,경도,city,gu,dong
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200606,1,60000,4,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동
1,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,20,72250,5,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동
2,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,30,73500,8,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동
3,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200612,9,76500,5,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동
4,서울특별시 강남구 개포동,658,1,개포6차우성아파트1동~8동,79.97,200605,9,88000,3,1987,언주로 3,37.476792,127.055926,서울특별시,강남구,개포동


In [398]:
# train 데이터 열이름 교체
train = train.rename(columns={'단지명':'apt','전용면적(㎡)':'exclusive_area',
                              '계약년월':'transaction_ym','계약일':'transaction_day','거래금액(만원)':'transaction_real_price',
                              '층':'floor','건축년도':'completion_year','도로명':'addr_kr','위도':'apt_lat','경도':'apt_lng'})
train.head()

,시군구,본번,부번,apt,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,addr_kr,apt_lat,apt_lng,city,gu,dong
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200606,1,60000,4,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동
1,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,20,72250,5,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동
2,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,30,73500,8,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동
3,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200612,9,76500,5,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동
4,서울특별시 강남구 개포동,658,1,개포6차우성아파트1동~8동,79.97,200605,9,88000,3,1987,언주로 3,37.476792,127.055926,서울특별시,강남구,개포동


In [399]:
# 법정동 코드를 동이름 기준으로 재구성

dong_code = pd.read_csv('data/ProcessedData/dong_code.csv', encoding='cp949')
dong_code

,법정동코드,시도명,시군구명,읍면동명,동리명,생성일자,말소일자
0,1111010100,서울특별시,종로구,청운동,NaN,19880423,NaN
1,1111010200,서울특별시,종로구,신교동,NaN,19880423,NaN
2,1111010300,서울특별시,종로구,궁정동,NaN,19880423,NaN
3,1111010400,서울특별시,종로구,효자동,NaN,19880423,NaN
4,1111010500,서울특별시,종로구,창성동,NaN,19880423,NaN
...,...,...,...,...,...,...,...
462,1174010600,서울특별시,강동구,둔촌동,NaN,19880423,NaN
463,1174010700,서울특별시,강동구,암사동,NaN,19880423,NaN
464,1174010800,서울특별시,강동구,성내동,NaN,19880423,NaN
465,1174010900,서울특별시,강동구,천호동,NaN,19880423,NaN


In [400]:
dong_code.읍면동명.value_counts()

신정동    2
신사동    2
우이동    1
정릉동    1
구기동    1
      ..
관수동    1
묵동     1
옥천동    1
장위동    1
서린동    1
Name: 읍면동명, Length: 465, dtype: int64

In [401]:
dong_code.query("읍면동명 == '신정동'")

,법정동코드,시도명,시군구명,읍면동명,동리명,생성일자,말소일자
336,1144011700,서울특별시,마포구,신정동,NaN,19880423,NaN
346,1147010100,서울특별시,양천구,신정동,NaN,19880423,NaN


In [402]:
dong_code.query("읍면동명 == '신사동'")

,법정동코드,시도명,시군구명,읍면동명,동리명,생성일자,말소일자
297,1138010900,서울특별시,은평구,신사동,NaN,19890901,NaN
436,1168010700,서울특별시,강남구,신사동,NaN,19880423,NaN


In [403]:
dong_code = dong_code[['법정동코드','시군구명','읍면동명']].sort_values(['시군구명','읍면동명'])
dong_code

,법정동코드,시군구명,읍면동명
432,1168010300,강남구,개포동
437,1168010800,강남구,논현동
435,1168010600,강남구,대치동
444,1168011800,강남구,도곡동
434,1168010500,강남구,삼성동
...,...,...,...
231,1126010100,중랑구,면목동
234,1126010400,중랑구,묵동
232,1126010200,중랑구,상봉동
236,1126010600,중랑구,신내동


In [404]:
dong_code.columns = ['code','gu','dong']
dong_code

,code,gu,dong
432,1168010300,강남구,개포동
437,1168010800,강남구,논현동
435,1168010600,강남구,대치동
444,1168011800,강남구,도곡동
434,1168010500,강남구,삼성동
...,...,...,...
231,1126010100,중랑구,면목동
234,1126010400,중랑구,묵동
232,1126010200,중랑구,상봉동
236,1126010600,중랑구,신내동


In [405]:
train = pd.merge(train,dong_code, how='left')
train

,시군구,본번,부번,apt,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,addr_kr,apt_lat,apt_lng,city,gu,dong,code
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200606,1,60000,4,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동,1168010300
1,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,20,72250,5,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동,1168010300
2,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,30,73500,8,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동,1168010300
3,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200612,9,76500,5,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동,1168010300
4,서울특별시 강남구 개포동,658,1,개포6차우성아파트1동~8동,79.97,200605,9,88000,3,1987,언주로 3,37.476792,127.055926,서울특별시,강남구,개포동,1168010300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825533,서울특별시 중랑구 중화동,450,0,한신2,59.76,201704,26,33000,22,1998,동일로 752,37.597009,127.082071,서울특별시,중랑구,중화동,1126010300
825534,서울특별시 중랑구 중화동,450,0,한신2,84.03,201705,1,40200,22,1998,동일로 752,37.597009,127.082071,서울특별시,중랑구,중화동,1126010300
825535,서울특별시 중랑구 중화동,274,51,한영(101),84.69,201605,17,30000,4,2003,동일로144길 74,37.606521,127.082165,서울특별시,중랑구,중화동,1126010300
825536,서울특별시 중랑구 중화동,274,76,한영(103),67.44,201610,25,25700,5,2003,동일로144길 74,37.606521,127.082165,서울특별시,중랑구,중화동,1126010300


In [406]:
train.corr()

,본번,부번,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,apt_lat,apt_lng,code
본번,1.000000,-0.042554,-0.017092,0.009055,-0.000744,-0.086207,0.032009,0.154751,-0.110996,-0.213438,-0.004996
부번,-0.042554,1.000000,0.013920,-0.012035,0.000199,0.000667,-0.034504,0.001434,-0.038617,-0.031718,-0.033576
exclusive_area,-0.017092,0.013920,1.000000,0.068094,0.004133,0.674036,0.094644,0.127407,-0.144324,-0.006101,0.059164
transaction_ym,0.009055,-0.012035,0.068094,1.000000,-0.004910,0.169293,0.041206,0.118865,-0.051246,-0.017836,0.019863
transaction_day,-0.000744,0.000199,0.004133,-0.004910,1.000000,0.008577,0.001104,0.000743,-0.002990,0.001859,0.003473
transaction_real_price,-0.086207,0.000667,0.674036,0.169293,0.008577,1.000000,0.134393,-0.018455,-0.322243,0.102185,0.292195
floor,0.032009,-0.034504,0.094644,0.041206,0.001104,0.134393,1.000000,0.186189,-0.037366,0.018239,-0.035854
completion_year,0.154751,0.001434,0.127407,0.118865,0.000743,-0.018455,0.186189,1.000000,-0.061931,-0.118306,-0.119143
apt_lat,-0.110996,-0.038617,-0.144324,-0.051246,-0.002990,-0.322243,-0.037366,-0.061931,1.000000,0.279964,-0.622560
apt_lng,-0.213438,-0.031718,-0.006101,-0.017836,0.001859,0.102185,0.018239,-0.118306,0.279964,1.000000,-0.013848


### 지하철 데이터 적용

In [407]:
subway = pd.read_csv('data/ProcessedData/subway_cnt.csv', encoding='cp949')
subway

,gu,subway_cnt
0,강남구,28
1,강동구,10
2,강북구,3
3,강서구,19
4,관악구,4
5,광진구,10
6,구로구,13
7,금천구,3
8,노원구,15
9,도봉구,7


In [408]:
train = pd.merge(train,subway, how='left')
train

,시군구,본번,부번,apt,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,addr_kr,apt_lat,apt_lng,city,gu,dong,code,subway_cnt
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200606,1,60000,4,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동,1168010300,28
1,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,20,72250,5,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동,1168010300,28
2,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,30,73500,8,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동,1168010300,28
3,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200612,9,76500,5,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동,1168010300,28
4,서울특별시 강남구 개포동,658,1,개포6차우성아파트1동~8동,79.97,200605,9,88000,3,1987,언주로 3,37.476792,127.055926,서울특별시,강남구,개포동,1168010300,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825533,서울특별시 중랑구 중화동,450,0,한신2,59.76,201704,26,33000,22,1998,동일로 752,37.597009,127.082071,서울특별시,중랑구,중화동,1126010300,11
825534,서울특별시 중랑구 중화동,450,0,한신2,84.03,201705,1,40200,22,1998,동일로 752,37.597009,127.082071,서울특별시,중랑구,중화동,1126010300,11
825535,서울특별시 중랑구 중화동,274,51,한영(101),84.69,201605,17,30000,4,2003,동일로144길 74,37.606521,127.082165,서울특별시,중랑구,중화동,1126010300,11
825536,서울특별시 중랑구 중화동,274,76,한영(103),67.44,201610,25,25700,5,2003,동일로144길 74,37.606521,127.082165,서울특별시,중랑구,중화동,1126010300,11


In [409]:
train.corr()

,본번,부번,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,apt_lat,apt_lng,code,subway_cnt
본번,1.000000,-0.042554,-0.017092,0.009055,-0.000744,-0.086207,0.032009,0.154751,-0.110996,-0.213438,-0.004996,-0.152331
부번,-0.042554,1.000000,0.013920,-0.012035,0.000199,0.000667,-0.034504,0.001434,-0.038617,-0.031718,-0.033576,-0.000082
exclusive_area,-0.017092,0.013920,1.000000,0.068094,0.004133,0.674036,0.094644,0.127407,-0.144324,-0.006101,0.059164,0.058745
transaction_ym,0.009055,-0.012035,0.068094,1.000000,-0.004910,0.169293,0.041206,0.118865,-0.051246,-0.017836,0.019863,0.033480
transaction_day,-0.000744,0.000199,0.004133,-0.004910,1.000000,0.008577,0.001104,0.000743,-0.002990,0.001859,0.003473,0.002644
transaction_real_price,-0.086207,0.000667,0.674036,0.169293,0.008577,1.000000,0.134393,-0.018455,-0.322243,0.102185,0.292195,0.356144
floor,0.032009,-0.034504,0.094644,0.041206,0.001104,0.134393,1.000000,0.186189,-0.037366,0.018239,-0.035854,-0.005786
completion_year,0.154751,0.001434,0.127407,0.118865,0.000743,-0.018455,0.186189,1.000000,-0.061931,-0.118306,-0.119143,-0.092118
apt_lat,-0.110996,-0.038617,-0.144324,-0.051246,-0.002990,-0.322243,-0.037366,-0.061931,1.000000,0.279964,-0.622560,-0.230133
apt_lng,-0.213438,-0.031718,-0.006101,-0.017836,0.001859,0.102185,0.018239,-0.118306,0.279964,1.000000,-0.013848,0.079069


#### 공원 데이터 적용

In [410]:
park = pd.read_csv('data/ProcessedData/park_cnt.csv', encoding='cp949')
park.head()

,gu,park_cnt
0,강남구,134
1,강동구,74
2,강북구,45
3,강서구,139
4,관악구,76


In [411]:
park.gu.nunique()

24

In [412]:
train = pd.merge(train,park, how='left')
train

,시군구,본번,부번,apt,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,addr_kr,apt_lat,apt_lng,city,gu,dong,code,subway_cnt,park_cnt
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200606,1,60000,4,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동,1168010300,28,134.0
1,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,20,72250,5,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동,1168010300,28,134.0
2,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,30,73500,8,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동,1168010300,28,134.0
3,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200612,9,76500,5,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동,1168010300,28,134.0
4,서울특별시 강남구 개포동,658,1,개포6차우성아파트1동~8동,79.97,200605,9,88000,3,1987,언주로 3,37.476792,127.055926,서울특별시,강남구,개포동,1168010300,28,134.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825533,서울특별시 중랑구 중화동,450,0,한신2,59.76,201704,26,33000,22,1998,동일로 752,37.597009,127.082071,서울특별시,중랑구,중화동,1126010300,11,54.0
825534,서울특별시 중랑구 중화동,450,0,한신2,84.03,201705,1,40200,22,1998,동일로 752,37.597009,127.082071,서울특별시,중랑구,중화동,1126010300,11,54.0
825535,서울특별시 중랑구 중화동,274,51,한영(101),84.69,201605,17,30000,4,2003,동일로144길 74,37.606521,127.082165,서울특별시,중랑구,중화동,1126010300,11,54.0
825536,서울특별시 중랑구 중화동,274,76,한영(103),67.44,201610,25,25700,5,2003,동일로144길 74,37.606521,127.082165,서울특별시,중랑구,중화동,1126010300,11,54.0


In [413]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 825538 entries, 0 to 825537
Data columns (total 19 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   시군구                     825538 non-null  object 
 1   본번                      825538 non-null  int64  
 2   부번                      825538 non-null  int64  
 3   apt                     825538 non-null  object 
 4   exclusive_area          825538 non-null  float64
 5   transaction_ym          825538 non-null  int64  
 6   transaction_day         825538 non-null  int64  
 7   transaction_real_price  825538 non-null  int64  
 8   floor                   825538 non-null  int64  
 9   completion_year         825538 non-null  int64  
 10  addr_kr                 825538 non-null  object 
 11  apt_lat                 803007 non-null  float64
 12  apt_lng                 803007 non-null  float64
 13  city                    825538 non-null  object 
 14  gu                  

In [414]:
train.park_cnt = train.park_cnt.fillna(0)
train

,시군구,본번,부번,apt,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,addr_kr,apt_lat,apt_lng,city,gu,dong,code,subway_cnt,park_cnt
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200606,1,60000,4,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동,1168010300,28,134.0
1,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,20,72250,5,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동,1168010300,28,134.0
2,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,30,73500,8,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동,1168010300,28,134.0
3,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200612,9,76500,5,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동,1168010300,28,134.0
4,서울특별시 강남구 개포동,658,1,개포6차우성아파트1동~8동,79.97,200605,9,88000,3,1987,언주로 3,37.476792,127.055926,서울특별시,강남구,개포동,1168010300,28,134.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825533,서울특별시 중랑구 중화동,450,0,한신2,59.76,201704,26,33000,22,1998,동일로 752,37.597009,127.082071,서울특별시,중랑구,중화동,1126010300,11,54.0
825534,서울특별시 중랑구 중화동,450,0,한신2,84.03,201705,1,40200,22,1998,동일로 752,37.597009,127.082071,서울특별시,중랑구,중화동,1126010300,11,54.0
825535,서울특별시 중랑구 중화동,274,51,한영(101),84.69,201605,17,30000,4,2003,동일로144길 74,37.606521,127.082165,서울특별시,중랑구,중화동,1126010300,11,54.0
825536,서울특별시 중랑구 중화동,274,76,한영(103),67.44,201610,25,25700,5,2003,동일로144길 74,37.606521,127.082165,서울특별시,중랑구,중화동,1126010300,11,54.0


In [415]:
train.isna().sum()

시군구                           0
본번                            0
부번                            0
apt                           0
exclusive_area                0
transaction_ym                0
transaction_day               0
transaction_real_price        0
floor                         0
completion_year               0
addr_kr                       0
apt_lat                   22531
apt_lng                   22531
city                          0
gu                            0
dong                          0
code                          0
subway_cnt                    0
park_cnt                      0
dtype: int64

In [416]:
train.corr()

,본번,부번,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,apt_lat,apt_lng,code,subway_cnt,park_cnt
본번,1.000000,-0.042554,-0.017092,0.009055,-0.000744,-0.086207,0.032009,0.154751,-0.110996,-0.213438,-0.004996,-0.152331,-0.051728
부번,-0.042554,1.000000,0.013920,-0.012035,0.000199,0.000667,-0.034504,0.001434,-0.038617,-0.031718,-0.033576,-0.000082,-0.015630
exclusive_area,-0.017092,0.013920,1.000000,0.068094,0.004133,0.674036,0.094644,0.127407,-0.144324,-0.006101,0.059164,0.058745,0.109343
transaction_ym,0.009055,-0.012035,0.068094,1.000000,-0.004910,0.169293,0.041206,0.118865,-0.051246,-0.017836,0.019863,0.033480,0.036119
transaction_day,-0.000744,0.000199,0.004133,-0.004910,1.000000,0.008577,0.001104,0.000743,-0.002990,0.001859,0.003473,0.002644,0.002973
transaction_real_price,-0.086207,0.000667,0.674036,0.169293,0.008577,1.000000,0.134393,-0.018455,-0.322243,0.102185,0.292195,0.356144,0.359395
floor,0.032009,-0.034504,0.094644,0.041206,0.001104,0.134393,1.000000,0.186189,-0.037366,0.018239,-0.035854,-0.005786,-0.042522
completion_year,0.154751,0.001434,0.127407,0.118865,0.000743,-0.018455,0.186189,1.000000,-0.061931,-0.118306,-0.119143,-0.092118,-0.042489
apt_lat,-0.110996,-0.038617,-0.144324,-0.051246,-0.002990,-0.322243,-0.037366,-0.061931,1.000000,0.279964,-0.622560,-0.230133,-0.492042
apt_lng,-0.213438,-0.031718,-0.006101,-0.017836,0.001859,0.102185,0.018239,-0.118306,0.279964,1.000000,-0.013848,0.079069,-0.103820


### 스타벅스 데이터 적용

In [460]:
starbucks = pd.read_csv('data/ProcessedData/starbucks_cnt.csv', encoding='cp949')
starbucks

,gu,starbucks_cnt
0,강남구,84
1,강동구,13
2,강북구,5
3,강서구,17
4,관악구,11
5,광진구,14
6,구로구,12
7,금천구,13
8,노원구,9
9,도봉구,1


In [461]:
train = pd.merge(train,starbucks, how='left')
train.head(1)

,시군구,본번,부번,apt,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,...,city,gu,dong,code,subway_cnt,park_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,starbucks_cnt
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200606,1,60000,4,1988,...,서울특별시,강남구,개포동,1168010300,28,134.0,6789,223.0,40,84


In [459]:
train.corr()

,본번,부번,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,apt_lat,apt_lng,code,subway_cnt,park_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,starbucks_cnt
본번,1.000000,-0.042554,-0.017092,0.009055,-0.000744,-0.086207,0.032009,0.154751,-0.110996,-0.213438,-0.004996,-0.152331,-0.051728,-0.053718,0.090220,-0.029372,-0.049022
부번,-0.042554,1.000000,0.013920,-0.012035,0.000199,0.000667,-0.034504,0.001434,-0.038617,-0.031718,-0.033576,-0.000082,-0.015630,-0.040926,-0.033528,-0.073319,0.018976
exclusive_area,-0.017092,0.013920,1.000000,0.068094,0.004133,0.674036,0.094644,0.127407,-0.144324,-0.006101,0.059164,0.058745,0.109343,0.068036,-0.123420,-0.151341,0.139974
transaction_ym,0.009055,-0.012035,0.068094,1.000000,-0.004910,0.169293,0.041206,0.118865,-0.051246,-0.017836,0.019863,0.033480,0.036119,0.012810,-0.015198,-0.028363,0.030067
transaction_day,-0.000744,0.000199,0.004133,-0.004910,1.000000,0.008577,0.001104,0.000743,-0.002990,0.001859,0.003473,0.002644,0.002973,0.003717,-0.002239,-0.001673,0.004402
transaction_real_price,-0.086207,0.000667,0.674036,0.169293,0.008577,1.000000,0.134393,-0.018455,-0.322243,0.102185,0.292195,0.356144,0.359395,0.431926,-0.124953,-0.161846,0.500434
floor,0.032009,-0.034504,0.094644,0.041206,0.001104,0.134393,1.000000,0.186189,-0.037366,0.018239,-0.035854,-0.005786,-0.042522,-0.050863,-0.068662,-0.043095,-0.009194
completion_year,0.154751,0.001434,0.127407,0.118865,0.000743,-0.018455,0.186189,1.000000,-0.061931,-0.118306,-0.119143,-0.092118,-0.042489,-0.173234,-0.107050,-0.121439,-0.100151
apt_lat,-0.110996,-0.038617,-0.144324,-0.051246,-0.002990,-0.322243,-0.037366,-0.061931,1.000000,0.279964,-0.622560,-0.230133,-0.492042,-0.300381,0.066685,0.381593,-0.443285
apt_lng,-0.213438,-0.031718,-0.006101,-0.017836,0.001859,0.102185,0.018239,-0.118306,0.279964,1.000000,-0.013848,0.079069,-0.103820,0.165593,-0.236038,0.177030,0.094022


#### 학원 데이터 적용

In [417]:
academy = pd.read_csv('data/ProcessedData/academy_cnt.csv', encoding='cp949')
academy

,gu,academy_cnt
0,강남구,6789
1,강동구,1914
2,강북구,654
3,강서구,2121
4,관악구,1350
5,광진구,1290
6,구로구,1281
7,금천구,666
8,노원구,2241
9,도봉구,1005


In [418]:
train = pd.merge(train,academy, how='left')
train

,시군구,본번,부번,apt,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,addr_kr,apt_lat,apt_lng,city,gu,dong,code,subway_cnt,park_cnt,academy_cnt
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200606,1,60000,4,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동,1168010300,28,134.0,6789
1,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,20,72250,5,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동,1168010300,28,134.0,6789
2,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,30,73500,8,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동,1168010300,28,134.0,6789
3,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200612,9,76500,5,1988,언주로 103,37.481344,127.052912,서울특별시,강남구,개포동,1168010300,28,134.0,6789
4,서울특별시 강남구 개포동,658,1,개포6차우성아파트1동~8동,79.97,200605,9,88000,3,1987,언주로 3,37.476792,127.055926,서울특별시,강남구,개포동,1168010300,28,134.0,6789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825533,서울특별시 중랑구 중화동,450,0,한신2,59.76,201704,26,33000,22,1998,동일로 752,37.597009,127.082071,서울특별시,중랑구,중화동,1126010300,11,54.0,819
825534,서울특별시 중랑구 중화동,450,0,한신2,84.03,201705,1,40200,22,1998,동일로 752,37.597009,127.082071,서울특별시,중랑구,중화동,1126010300,11,54.0,819
825535,서울특별시 중랑구 중화동,274,51,한영(101),84.69,201605,17,30000,4,2003,동일로144길 74,37.606521,127.082165,서울특별시,중랑구,중화동,1126010300,11,54.0,819
825536,서울특별시 중랑구 중화동,274,76,한영(103),67.44,201610,25,25700,5,2003,동일로144길 74,37.606521,127.082165,서울특별시,중랑구,중화동,1126010300,11,54.0,819


In [419]:
train.corr()

,본번,부번,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,apt_lat,apt_lng,code,subway_cnt,park_cnt,academy_cnt
본번,1.000000,-0.042554,-0.017092,0.009055,-0.000744,-0.086207,0.032009,0.154751,-0.110996,-0.213438,-0.004996,-0.152331,-0.051728,-0.053718
부번,-0.042554,1.000000,0.013920,-0.012035,0.000199,0.000667,-0.034504,0.001434,-0.038617,-0.031718,-0.033576,-0.000082,-0.015630,-0.040926
exclusive_area,-0.017092,0.013920,1.000000,0.068094,0.004133,0.674036,0.094644,0.127407,-0.144324,-0.006101,0.059164,0.058745,0.109343,0.068036
transaction_ym,0.009055,-0.012035,0.068094,1.000000,-0.004910,0.169293,0.041206,0.118865,-0.051246,-0.017836,0.019863,0.033480,0.036119,0.012810
transaction_day,-0.000744,0.000199,0.004133,-0.004910,1.000000,0.008577,0.001104,0.000743,-0.002990,0.001859,0.003473,0.002644,0.002973,0.003717
transaction_real_price,-0.086207,0.000667,0.674036,0.169293,0.008577,1.000000,0.134393,-0.018455,-0.322243,0.102185,0.292195,0.356144,0.359395,0.431926
floor,0.032009,-0.034504,0.094644,0.041206,0.001104,0.134393,1.000000,0.186189,-0.037366,0.018239,-0.035854,-0.005786,-0.042522,-0.050863
completion_year,0.154751,0.001434,0.127407,0.118865,0.000743,-0.018455,0.186189,1.000000,-0.061931,-0.118306,-0.119143,-0.092118,-0.042489,-0.173234
apt_lat,-0.110996,-0.038617,-0.144324,-0.051246,-0.002990,-0.322243,-0.037366,-0.061931,1.000000,0.279964,-0.622560,-0.230133,-0.492042,-0.300381
apt_lng,-0.213438,-0.031718,-0.006101,-0.017836,0.001859,0.102185,0.018239,-0.118306,0.279964,1.000000,-0.013848,0.079069,-0.103820,0.165593


#### 어린이집 데이터 적용

In [420]:
daycare = pd.read_csv('data/ProcessedData/daycare_center_cnt.csv', encoding='cp949')
daycare

,gu,daycare_cnt
0,강남구,223
1,강동구,272
2,강북구,167
3,강서구,874
4,관악구,271
5,광진구,197
6,구로구,334
7,금천구,168
8,노원구,457
9,도봉구,255


In [421]:
train.gu.nunique()

25

In [422]:
# 동대문구, 마포구, 성북구, 용산구 데이터가 없음
daycare.sort_values('gu').gu.nunique()

21

In [423]:
train = pd.merge(train,daycare, how='left')
train

,시군구,본번,부번,apt,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,...,apt_lat,apt_lng,city,gu,dong,code,subway_cnt,park_cnt,academy_cnt,daycare_cnt
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200606,1,60000,4,1988,...,37.481344,127.052912,서울특별시,강남구,개포동,1168010300,28,134.0,6789,223.0
1,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,20,72250,5,1988,...,37.481344,127.052912,서울특별시,강남구,개포동,1168010300,28,134.0,6789,223.0
2,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,30,73500,8,1988,...,37.481344,127.052912,서울특별시,강남구,개포동,1168010300,28,134.0,6789,223.0
3,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200612,9,76500,5,1988,...,37.481344,127.052912,서울특별시,강남구,개포동,1168010300,28,134.0,6789,223.0
4,서울특별시 강남구 개포동,658,1,개포6차우성아파트1동~8동,79.97,200605,9,88000,3,1987,...,37.476792,127.055926,서울특별시,강남구,개포동,1168010300,28,134.0,6789,223.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825533,서울특별시 중랑구 중화동,450,0,한신2,59.76,201704,26,33000,22,1998,...,37.597009,127.082071,서울특별시,중랑구,중화동,1126010300,11,54.0,819,242.0
825534,서울특별시 중랑구 중화동,450,0,한신2,84.03,201705,1,40200,22,1998,...,37.597009,127.082071,서울특별시,중랑구,중화동,1126010300,11,54.0,819,242.0
825535,서울특별시 중랑구 중화동,274,51,한영(101),84.69,201605,17,30000,4,2003,...,37.606521,127.082165,서울특별시,중랑구,중화동,1126010300,11,54.0,819,242.0
825536,서울특별시 중랑구 중화동,274,76,한영(103),67.44,201610,25,25700,5,2003,...,37.606521,127.082165,서울특별시,중랑구,중화동,1126010300,11,54.0,819,242.0


In [424]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 825538 entries, 0 to 825537
Data columns (total 21 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   시군구                     825538 non-null  object 
 1   본번                      825538 non-null  int64  
 2   부번                      825538 non-null  int64  
 3   apt                     825538 non-null  object 
 4   exclusive_area          825538 non-null  float64
 5   transaction_ym          825538 non-null  int64  
 6   transaction_day         825538 non-null  int64  
 7   transaction_real_price  825538 non-null  int64  
 8   floor                   825538 non-null  int64  
 9   completion_year         825538 non-null  int64  
 10  addr_kr                 825538 non-null  object 
 11  apt_lat                 803007 non-null  float64
 12  apt_lng                 803007 non-null  float64
 13  city                    825538 non-null  object 
 14  gu                  

In [425]:
train.daycare_cnt = train.daycare_cnt.fillna(0)
train

,시군구,본번,부번,apt,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,...,apt_lat,apt_lng,city,gu,dong,code,subway_cnt,park_cnt,academy_cnt,daycare_cnt
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200606,1,60000,4,1988,...,37.481344,127.052912,서울특별시,강남구,개포동,1168010300,28,134.0,6789,223.0
1,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,20,72250,5,1988,...,37.481344,127.052912,서울특별시,강남구,개포동,1168010300,28,134.0,6789,223.0
2,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,30,73500,8,1988,...,37.481344,127.052912,서울특별시,강남구,개포동,1168010300,28,134.0,6789,223.0
3,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200612,9,76500,5,1988,...,37.481344,127.052912,서울특별시,강남구,개포동,1168010300,28,134.0,6789,223.0
4,서울특별시 강남구 개포동,658,1,개포6차우성아파트1동~8동,79.97,200605,9,88000,3,1987,...,37.476792,127.055926,서울특별시,강남구,개포동,1168010300,28,134.0,6789,223.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825533,서울특별시 중랑구 중화동,450,0,한신2,59.76,201704,26,33000,22,1998,...,37.597009,127.082071,서울특별시,중랑구,중화동,1126010300,11,54.0,819,242.0
825534,서울특별시 중랑구 중화동,450,0,한신2,84.03,201705,1,40200,22,1998,...,37.597009,127.082071,서울특별시,중랑구,중화동,1126010300,11,54.0,819,242.0
825535,서울특별시 중랑구 중화동,274,51,한영(101),84.69,201605,17,30000,4,2003,...,37.606521,127.082165,서울특별시,중랑구,중화동,1126010300,11,54.0,819,242.0
825536,서울특별시 중랑구 중화동,274,76,한영(103),67.44,201610,25,25700,5,2003,...,37.606521,127.082165,서울특별시,중랑구,중화동,1126010300,11,54.0,819,242.0


In [426]:
train.corr()

,본번,부번,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,apt_lat,apt_lng,code,subway_cnt,park_cnt,academy_cnt,daycare_cnt
본번,1.000000,-0.042554,-0.017092,0.009055,-0.000744,-0.086207,0.032009,0.154751,-0.110996,-0.213438,-0.004996,-0.152331,-0.051728,-0.053718,0.090220
부번,-0.042554,1.000000,0.013920,-0.012035,0.000199,0.000667,-0.034504,0.001434,-0.038617,-0.031718,-0.033576,-0.000082,-0.015630,-0.040926,-0.033528
exclusive_area,-0.017092,0.013920,1.000000,0.068094,0.004133,0.674036,0.094644,0.127407,-0.144324,-0.006101,0.059164,0.058745,0.109343,0.068036,-0.123420
transaction_ym,0.009055,-0.012035,0.068094,1.000000,-0.004910,0.169293,0.041206,0.118865,-0.051246,-0.017836,0.019863,0.033480,0.036119,0.012810,-0.015198
transaction_day,-0.000744,0.000199,0.004133,-0.004910,1.000000,0.008577,0.001104,0.000743,-0.002990,0.001859,0.003473,0.002644,0.002973,0.003717,-0.002239
transaction_real_price,-0.086207,0.000667,0.674036,0.169293,0.008577,1.000000,0.134393,-0.018455,-0.322243,0.102185,0.292195,0.356144,0.359395,0.431926,-0.124953
floor,0.032009,-0.034504,0.094644,0.041206,0.001104,0.134393,1.000000,0.186189,-0.037366,0.018239,-0.035854,-0.005786,-0.042522,-0.050863,-0.068662
completion_year,0.154751,0.001434,0.127407,0.118865,0.000743,-0.018455,0.186189,1.000000,-0.061931,-0.118306,-0.119143,-0.092118,-0.042489,-0.173234,-0.107050
apt_lat,-0.110996,-0.038617,-0.144324,-0.051246,-0.002990,-0.322243,-0.037366,-0.061931,1.000000,0.279964,-0.622560,-0.230133,-0.492042,-0.300381,0.066685
apt_lng,-0.213438,-0.031718,-0.006101,-0.017836,0.001859,0.102185,0.018239,-0.118306,0.279964,1.000000,-0.013848,0.079069,-0.103820,0.165593,-0.236038


#### 유치원 데이터 적용

In [427]:
kgarden = pd.read_csv('data/ProcessedData/kindergarden_cnt.csv', encoding='cp949')
kgarden

,gu,kindergargen_cnt
0,강남구,40
1,강동구,39
2,강북구,24
3,강서구,46
4,관악구,36
5,광진구,37
6,구로구,38
7,금천구,19
8,노원구,70
9,도봉구,31


In [428]:
train = pd.merge(train,kgarden, how='left')
train

,시군구,본번,부번,apt,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,...,apt_lng,city,gu,dong,code,subway_cnt,park_cnt,academy_cnt,daycare_cnt,kindergargen_cnt
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200606,1,60000,4,1988,...,127.052912,서울특별시,강남구,개포동,1168010300,28,134.0,6789,223.0,40
1,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,20,72250,5,1988,...,127.052912,서울특별시,강남구,개포동,1168010300,28,134.0,6789,223.0,40
2,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200610,30,73500,8,1988,...,127.052912,서울특별시,강남구,개포동,1168010300,28,134.0,6789,223.0,40
3,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200612,9,76500,5,1988,...,127.052912,서울특별시,강남구,개포동,1168010300,28,134.0,6789,223.0,40
4,서울특별시 강남구 개포동,658,1,개포6차우성아파트1동~8동,79.97,200605,9,88000,3,1987,...,127.055926,서울특별시,강남구,개포동,1168010300,28,134.0,6789,223.0,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825533,서울특별시 중랑구 중화동,450,0,한신2,59.76,201704,26,33000,22,1998,...,127.082071,서울특별시,중랑구,중화동,1126010300,11,54.0,819,242.0,32
825534,서울특별시 중랑구 중화동,450,0,한신2,84.03,201705,1,40200,22,1998,...,127.082071,서울특별시,중랑구,중화동,1126010300,11,54.0,819,242.0,32
825535,서울특별시 중랑구 중화동,274,51,한영(101),84.69,201605,17,30000,4,2003,...,127.082165,서울특별시,중랑구,중화동,1126010300,11,54.0,819,242.0,32
825536,서울특별시 중랑구 중화동,274,76,한영(103),67.44,201610,25,25700,5,2003,...,127.082165,서울특별시,중랑구,중화동,1126010300,11,54.0,819,242.0,32


In [70]:
train.isna().sum()

시군구                       0
본번                        0
부번                        0
apt                       0
exclusive_area            0
transaction_ym            0
transaction_day           0
transaction_real_price    0
floor                     0
completion_year           0
addr_kr                   0
apt_lat                   0
apt_lng                   0
city                      0
gu                        0
dong                      0
code                      0
subway_cnt                0
park_cnt                  0
starbucks_cnt             0
academy_cnt               0
daycare_cnt               0
kindergarden_cnt          0
dist_river                0
dist_subway               0
dist_kgarden              0
dist_element              0
dist_middle               0
dist_high                 0
dist_starbucks            0
dtype: int64

In [430]:
train.corr()

,본번,부번,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,apt_lat,apt_lng,code,subway_cnt,park_cnt,academy_cnt,daycare_cnt,kindergargen_cnt
본번,1.000000,-0.042554,-0.017092,0.009055,-0.000744,-0.086207,0.032009,0.154751,-0.110996,-0.213438,-0.004996,-0.152331,-0.051728,-0.053718,0.090220,-0.029372
부번,-0.042554,1.000000,0.013920,-0.012035,0.000199,0.000667,-0.034504,0.001434,-0.038617,-0.031718,-0.033576,-0.000082,-0.015630,-0.040926,-0.033528,-0.073319
exclusive_area,-0.017092,0.013920,1.000000,0.068094,0.004133,0.674036,0.094644,0.127407,-0.144324,-0.006101,0.059164,0.058745,0.109343,0.068036,-0.123420,-0.151341
transaction_ym,0.009055,-0.012035,0.068094,1.000000,-0.004910,0.169293,0.041206,0.118865,-0.051246,-0.017836,0.019863,0.033480,0.036119,0.012810,-0.015198,-0.028363
transaction_day,-0.000744,0.000199,0.004133,-0.004910,1.000000,0.008577,0.001104,0.000743,-0.002990,0.001859,0.003473,0.002644,0.002973,0.003717,-0.002239,-0.001673
transaction_real_price,-0.086207,0.000667,0.674036,0.169293,0.008577,1.000000,0.134393,-0.018455,-0.322243,0.102185,0.292195,0.356144,0.359395,0.431926,-0.124953,-0.161846
floor,0.032009,-0.034504,0.094644,0.041206,0.001104,0.134393,1.000000,0.186189,-0.037366,0.018239,-0.035854,-0.005786,-0.042522,-0.050863,-0.068662,-0.043095
completion_year,0.154751,0.001434,0.127407,0.118865,0.000743,-0.018455,0.186189,1.000000,-0.061931,-0.118306,-0.119143,-0.092118,-0.042489,-0.173234,-0.107050,-0.121439
apt_lat,-0.110996,-0.038617,-0.144324,-0.051246,-0.002990,-0.322243,-0.037366,-0.061931,1.000000,0.279964,-0.622560,-0.230133,-0.492042,-0.300381,0.066685,0.381593
apt_lng,-0.213438,-0.031718,-0.006101,-0.017836,0.001859,0.102185,0.018239,-0.118306,0.279964,1.000000,-0.013848,0.079069,-0.103820,0.165593,-0.236038,0.177030


#### 초,중,고,특수학교 데이터 적용

In [289]:
#### 초,중,고,특수학교의 구별 빈도수는 어린이집,유치원,초중고(+특수)끼리는 관계가 분명하게 있지만, 아파트거래금액에는 영향이 거의 없음
#### 그러므로 구별 초중고(+특수) 각각의 학교위치와 구별 아파트위치의 거리를 계산하여 적용 시켜보기로함

## 초등학교 데이터 적용
#element = pd.read_csv('data/ProcessedData/element_cnt.csv', encoding='cp949')
#element
#train = pd.merge(train,element, how='left')
#train

## 중학교 데이터 적용
#middle = pd.read_csv('data/ProcessedData/middle_cnt.csv', encoding='cp949')
#middle
#train = pd.merge(train,middle, how='left')
#train

## 고등학교 데이터 적용
#high = pd.read_csv('data/ProcessedData/high_cnt.csv', encoding='cp949')
#high
#train = pd.merge(train,high, how='left')
#train

## 특수학교 데이터 적용
#special = pd.read_csv('data/ProcessedData/special_cnt.csv', encoding='cp949')
#special
#train = pd.merge(train,special, how='left')
#train

#train.corr() # 아파트 거래금액과의 상관관계 계수 : -0.03,0.08,-0.01,0.01

# 아파트데이터 추가전처리 및 파생변수 생성과정 함수화

In [506]:
def data_cleaning(path):
    
    import pandas as pd
    import numpy as np
    import time
    
    start = time.time()
    data = pd.read_csv(path, encoding='cp949')

    # 거래금액열에 값들의 ','를 없애기
    data['거래금액(만원)'] = data['거래금액(만원)'].str.replace(',','')
    data['거래금액(만원)'] = data['거래금액(만원)'].astype(int)
    print("========== COMPLETED REPLACE")

    # data 의 시군구 컬럼에서 구이름과 동이름만 가져오기
    data['city'], data['gu'], data['dong'] = data['시군구'].str.split(' ',2).str
    print("========== COMPLETED SPLIT")
        
    # 열이름 영어이름으로 바꿔주기
    data = data.rename(columns={'단지명':'apt','전용면적(㎡)':'exclusive_area',
                                  '계약년월':'transaction_ym','계약일':'transaction_day','거래금액(만원)':'transaction_real_price',
                                  '층':'floor','건축년도':'completion_year','도로명':'addr_kr','위도':'apt_lat','경도':'apt_lng'})
    print("========== COMPLETED RENAME")
    ##### 법정동코드 데이터 적용 #####
    # 법정동 코드 데이터 불러오기
    dong_code = pd.read_csv('data/ProcessedData/dong_code.csv', encoding='cp949')
    # 법정동 코드를 동이름 기준으로 재구성
    dong_code = dong_code[['법정동코드','시군구명','읍면동명']].sort_values(['시군구명','읍면동명'])
    # 열이름 교체
    dong_code.columns = ['code','gu','dong']
    # 데이터에 법정동코드 적용
    data = pd.merge(data,dong_code, how='left')
    print("========== COMPLETED APPLY DONGCODE")
    
    ##### 지하철 데이터 적용 #####
    # 지하철 데이터 불러오기
    subway = pd.read_csv('data/ProcessedData/subway_cnt.csv', encoding='cp949')
    # 데이터에 적용
    data = pd.merge(data,subway, how='left')
    print("========== COMPLETED APPLY SUBWAY")
    
    ##### 공원 데이터 적용  #####
    # 공원 데이터 불러오기
    park = pd.read_csv('data/ProcessedData/park_cnt.csv', encoding='cp949')
    # 데이터에 적용
    data = pd.merge(data,park, how='left')
    # 'park_cnt' 결측치 대체
    data.park_cnt = data.park_cnt.fillna(0)
    print("========== COMPLETED APPLY PARK COUNT DATA")
    
    ##### 스타벅스 데이터 적용 #####
    # 스타벅스 데이터 불러오기
    starbucks = pd.read_csv('data/ProcessedData/starbucks_cnt.csv', encoding='cp949')
    # 데이터 적용
    data = pd.merge(data,starbucks, how='left')
    print("========== COMPLETED APPLY STARBUCKS COUNT DATA")
    
    ##### 학원 데이터 적용 #####
    academy = pd.read_csv('data/ProcessedData/academy_cnt.csv', encoding='cp949')
    # 데이터에 적용
    data = pd.merge(data,academy, how='left')
    print("========== COMPLETED APPLY ACADEMY COUNT DATA")
    
    ##### 어린이집 데이터 적용  #####
    # 어린이집 데이터 불러오기
    daycare = pd.read_csv('data/ProcessedData/daycare_center_cnt.csv', encoding='cp949')
    # 데이터에 적용
    data = pd.merge(data,daycare, how='left')
    # 'daycare_cnt' 결측치 대체
    data.daycare_cnt = data.daycare_cnt.fillna(0)
    print("========== COMPLETED APPLY DAY CARE CENTER COUNT DATA")
    
    ##### 유치원 데이터 전처리 #####
    kgarden = pd.read_csv('data/ProcessedData/kindergarden_cnt.csv', encoding='cp949')
    data = pd.merge(data,kgarden, how='left')
    print("========== COMPLETED APPLY KINDERGARDEN COUNT DATA")
    print('---------- Done {:.2f} sec ----------'.format(time.time() - start))
    
    #### 초,중,고,특수학교의 구별 빈도수는 어린이집,유치원,초중고(+특수)끼리는 관계가 분명하게 있지만, 아파트거래금액에는 영향이 거의 없음
    #### 그러므로 구별 초중고(+특수) 각각의 학교위치와 구별 아파트위치의 거리를 계산하여 적용 시켜보기로함
    ## 초등학교 데이터 적용 ##
    # element = pd.read_csv('data/ProcessedData/element_cnt.csv', encoding='cp949')
    # train = pd.merge(train,element, how='left')
    ## 중학교 데이터 적용 ##
    # middle = pd.read_csv('data/ProcessedData/middle_cnt.csv', encoding='cp949')
    # train = pd.merge(train,middle, how='left')
    ## 고등학교 데이터 적용 ##
    # high = pd.read_csv('data/ProcessedData/high_cnt.csv', encoding='cp949')
    # train = pd.merge(train,high, how='left')
    ## 특수학교 데이터 적용 ##
    # special = pd.read_csv('data/ProcessedData/special_cnt.csv', encoding='cp949')
    # train = pd.merge(train,special, how='left')
    # train.corr() # 아파트 거래금액과의 상관관계 계수 : -0.03,0.08,-0.01,0.01
    
    return data

In [507]:
train = data_cleaning('data/ProcessedData/train_final.csv')

========== COMPLETED REPLACE


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  app.launch_new_instance()


========== COMPLETED SPLIT
========== COMPLETED RENAME
========== COMPLETED APPLY DONGCODE
========== COMPLETED APPLY SUBWAY
========== COMPLETED APPLY PARK COUNT DATA
========== COMPLETED APPLY STARBUCKS COUNT DATA
========== COMPLETED APPLY ACADEMY COUNT DATA
========== COMPLETED APPLY DAY CARE CENTER COUNT DATA
========== COMPLETED APPLY KINDERGARDEN COUNT DATA
---------- Done 11.08 sec ----------


In [508]:
train.head(1).T

,0
시군구,서울특별시 강남구 개포동
본번,655
부번,2
apt,개포2차현대아파트(220)
exclusive_area,77.75
transaction_ym,200606
transaction_day,1
transaction_real_price,60000
floor,4
completion_year,1988


In [509]:
test = data_cleaning('data/ProcessedData/test_final.csv')

========== COMPLETED REPLACE


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  app.launch_new_instance()


========== COMPLETED SPLIT
========== COMPLETED RENAME
========== COMPLETED APPLY DONGCODE
========== COMPLETED APPLY SUBWAY
========== COMPLETED APPLY PARK COUNT DATA
========== COMPLETED APPLY STARBUCKS COUNT DATA
========== COMPLETED APPLY ACADEMY COUNT DATA
========== COMPLETED APPLY DAY CARE CENTER COUNT DATA
========== COMPLETED APPLY KINDERGARDEN COUNT DATA
---------- Done 3.04 sec ----------


In [510]:
test.head(1).T

,0
시군구,서울특별시 강남구 개포동
본번,655
부번,2
apt,개포2차현대아파트(220)
exclusive_area,77.75
transaction_ym,201707
transaction_day,8
transaction_real_price,89900
floor,1
completion_year,1988


In [511]:
duplicate_tr = train.copy()
duplicate_tr = duplicate_tr[['gu','dong','addr_kr','apt','apt_lat','apt_lng']].drop_duplicates()
duplicate_tr

,gu,dong,addr_kr,apt,apt_lat,apt_lng
0,강남구,개포동,언주로 103,개포2차현대아파트(220),37.481344,127.052912
4,강남구,개포동,언주로 3,개포6차우성아파트1동~8동,37.476792,127.055926
11,강남구,개포동,개포로 307,개포우성3차,37.484631,127.056092
14,강남구,개포동,개포로109길 69,개포자이(12-2),37.496292,127.076127
18,강남구,개포동,삼성로 11,개포주공 3단지,37.484135,127.068092
...,...,...,...,...,...,...
799514,중구,황학동,난계로13길 36,로얄,37.567290,127.021571
800332,중랑구,면목동,사가정로42길 90,두산5,37.577959,127.081093
800651,중랑구,면목동,동일로101길 63,한신,37.588544,127.076312
801479,중랑구,상봉동,상봉로 131,상봉 듀오트리스,37.597407,127.092178


In [512]:
duplicate_te = test.copy()
duplicate_te = duplicate_te[['gu','dong','addr_kr','apt','apt_lat','apt_lng']].drop_duplicates()
duplicate_te

,gu,dong,addr_kr,apt,apt_lat,apt_lng
0,강남구,개포동,언주로 103,개포2차현대아파트(220),37.482630,127.051104
1,강남구,개포동,언주로 3,개포6차우성아파트1동~8동,37.476658,127.056909
16,강남구,개포동,개포로 307,개포우성3차,37.484141,127.055247
33,강남구,개포동,개포로109길 69,개포자이(12-2),37.496094,127.076466
45,강남구,개포동,개포로 310,개포주공 1단지,37.476834,127.058210
...,...,...,...,...,...,...
254329,중랑구,신내동,망우로65길 16,조호그레이스2,37.601096,127.099935
254505,중랑구,중화동,망우로 231-8,미영리치타운103,37.595202,127.078436
254546,중랑구,중화동,봉화산로27길 30,한솔e(A동),37.603255,127.083636
254548,중랑구,중화동,봉화산로27길 30,한솔e(B동),37.603255,127.083636


### 특정 장소와 거리구하기 함수화

In [513]:
def calculate_distance(data):
    
    import time
    
    func_start = time.time()
    ##### 자료 준비 #####
    import pandas as pd
    river = pd.read_csv('data/ProcessedData/location/river_loc.csv', encoding='cp949')
    subway = pd.read_csv('data/ProcessedData/location/subway_loc.csv', encoding='cp949')
    park = pd.read_csv('data/ProcessedData/location/park_loc.csv', encoding='cp949')
    starbucks = pd.read_csv('data/ProcessedData/location/starbucks_loc.csv', encoding='cp949')
    '''kgarden = pd.read_csv('data/ProcessedData/location/kindergarden_loc.csv', encoding='cp949')
    element = pd.read_csv('data/ProcessedData/location/element_loc.csv', encoding='cp949')
    middle = pd.read_csv('data/ProcessedData/location/middle_loc.csv', encoding='cp949')
    high = pd.read_csv('data/ProcessedData/location/high_loc.csv', encoding='cp949')'''
    print("data is ready to calculate")
    
    ##### 거리계산 함수 #####
    from math import cos, asin, sqrt
    def distance(lat1, lon1, lat2, lon2):
        p = 0.017453292519943295     #Pi/180
        a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
        return 12742 * asin(sqrt(a)) #2*R*asin...
    print("func of distance is working.....")
    
    ##### 거리열 미리 추가 #####
    import numpy as np
    data['dist_river'] = np.nan
    data['dist_subway'] = np.nan
    data['dist_park'] = np.nan
    data['dist_starbucks'] = np.nan
    '''data['dist_kgarden'] = np.nan
    data['dist_element'] = np.nan
    data['dist_middle'] = np.nan
    data['dist_high'] = np.nan'''
    print("each of distance columns are created")
    
    start = time.time()
    # < 1 > 한강과의 최단 거리
    for i in range(len(data)):
        for j in range(len(river)):
            # 아파트 위치와 한강의 거리를 구한다.
            dist = distance(data['apt_lat'].iloc[i],data['apt_lng'].iloc[i],river['lat'].iloc[j],river['lng'].iloc[j])
            if dist < data['dist_river'].iloc[i]:           # 새로 구한 거리가 기존의 거리보다 짧을 때
                data['dist_river'].iloc[i] = dist           # 새로 구한 거리로 대체
            elif dist > data['dist_river'].iloc[i]:         # 새로 구한 거리가 기존의 거리보다 길 때
                continue                                    # 다른 거리를 구한다.
            data['dist_river'].iloc[i] = dist               # <--기준점 거리(처음 구해진 거리)
    print("========== COMPLETED DINTANCE OF 'HAN'RIVER BETWEEN APT")
    print('---------- Done {:.2f} sec ----------'.format(time.time() - start))
    
    start = time.time()
    # < 2 > 지하철과의 최단 거리
    for i in range(len(data)):
        for j in range(len(subway)):
            # 아파트 위치와 지하철 위치의 거리를 구한다.
            dist = distance(data['apt_lat'].iloc[i],data['apt_lng'].iloc[i],subway['lat'].iloc[j],subway['lng'].iloc[j])
            if dist < data['dist_subway'].iloc[i]:          # 새로 구한 거리가 기존의 거리보다 짧을 때
                data['dist_subway'].iloc[i] = dist          # 새로 구한 거리로 대체
            elif dist > data['dist_subway'].iloc[i]:        # 새로 구한 거리가 기존의 거리보다 길 때
                continue                                    # 다른 거리를 구한다.
            data['dist_subway'].iloc[i] = dist              # <--기준점 거리(처음 구해진 거리)
    print("========== COMPLETED DINTANCE OF SUBWAY BETWEEN APT")
    print('---------- Done {:.2f} sec ----------'.format(time.time() - start))
    
    # < 3 > 공원과의 최단 거리
    start = time.time()
    for i in range(len(data)):
        for j in range(len(park)):
            # 아파트 위치와 공원의 거리를 구한다.
            dist = distance(data['apt_lat'].iloc[i],data['apt_lng'].iloc[i],park['lat'].iloc[j],park['lng'].iloc[j])
            if dist < data['dist_park'].iloc[i]:            # 새로 구한 거리가 기존의 거리보다 짧을 때
                data['dist_park'].iloc[i] = dist            # 새로 구한 거리로 대체
            elif dist > data['dist_park'].iloc[i]:          # 새로 구한 거리가 기존의 거리보다 길 때
                continue                                    # 다른 거리를 구한다.
            data['dist_park'].iloc[i] = dist           # <--기준점 거리(처음 구해진 거리)
    print("========== COMPLETED DINTANCE OF PARK BETWEEN APT")
    print('---------- Done {:.2f} sec ----------'.format(time.time() - start))
    
    start = time.time()
    # < 4 > 스타벅스와의 최단 거리
    for i in range(len(data)):
        for j in range(len(starbucks)):
            if data['gu'].iloc[i] == starbucks['gu'].iloc[j]:
                # 아파트 위치와 스타벅스 위치의 거리를 구한다.
                dist = distance(data['apt_lat'].iloc[i],data['apt_lng'].iloc[i],starbucks['lat'].iloc[j],starbucks['lng'].iloc[j])
                if dist < data['dist_starbucks'].iloc[i]:       # 새로 구한 거리가 기존의 거리보다 짧을 때
                    data['dist_starbucks'].iloc[i] = dist       # 새로 구한 거리로 대체
                elif dist > data['dist_starbucks'].iloc[i]:     # 새로 구한 거리가 기존의 거리보다 길 때
                    continue                                    # 다른 거리를 구한다.
                data['dist_starbucks'].iloc[i] = dist           # <--기준점 거리(처음 구해진 거리)
    print("========== COMPLETED DINTANCE OF STARBUCKS BETWEEN APT")
    print('---------- Done {:.2f} sec ----------'.format(time.time() - start))
    print('---------- Done All Function {:.2f} sec ----------'.format(time.time() - func_start))
    
    '''start = time.time()
    # < 5 > 유치원과의 최단 거리
    for i in range(len(data)):
        for j in range(len(kgarden)):
            if data['gu'].iloc[i] == kgarden['gu'].iloc[j]:
                # 아파트 위치와 유치원 위치의 거리를 구한다.
                dist = distance(data['apt_lat'].iloc[i],data['apt_lng'].iloc[i],kgarden['lat'].iloc[j],kgarden['lng'].iloc[j])
                if dist < data['dist_kgarden'].iloc[i]:    # 새로 구한 거리가 기존의 거리보다 짧을 때
                    data['dist_kgarden'].iloc[i] = dist       # 새로 구한 거리로 대체
                elif dist > data['dist_kgarden'].iloc[i]:  # 새로 구한 거리가 기존의 거리보다 길 때
                    continue                             # 다른 거리를 구한다.
                data['dist_kgarden'].iloc[i] = dist # <--기준점 거리(처음 구해진 거리)
    print("========== COMPLETED DINTANCE OF KINDERGARDEN BETWEEN APT")
    print('---------- Done {:.2f} sec ----------'.format(time.time() - start))
    
    start = time.time()
    # < 6 > 초등학교와의 최단 거리
    for i in range(len(data)):
        for j in range(len(element)):
            if data['gu'].iloc[i] == element['gu'].iloc[j]:
                # 아파트 위치와 초등학교 위치의 거리를 구한다.
                dist = distance(data['apt_lat'].iloc[i],data['apt_lng'].iloc[i],element['lat'].iloc[j],element['lng'].iloc[j])
                if dist < data['dist_element'].iloc[i]:    # 새로 구한 거리가 기존의 거리보다 짧을 때
                    data['dist_element'].iloc[i] = dist       # 새로 구한 거리로 대체
                elif dist > data['dist_element'].iloc[i]:  # 새로 구한 거리가 기존의 거리보다 길 때
                    continue                             # 다른 거리를 구한다.
                data['dist_element'].iloc[i] = dist # <--기준점 거리(처음 구해진 거리)
    print("========== COMPLETED DINTANCE OF ELEMENTARY SCHOOL BETWEEN APT")
    print('---------- Done {:.2f} sec ----------'.format(time.time() - start))
    
    start = time.time()
    # < 7 > 중학교와의 최단 거리
    for i in range(len(data)):
        for j in range(len(middle)):
            if data['gu'].iloc[i] == middle['gu'].iloc[j]:
                # 아파트 위치와 중학교 위치의 거리를 구한다.
                dist = distance(data['apt_lat'].iloc[i],data['apt_lng'].iloc[i],middle['lat'].iloc[j],middle['lng'].iloc[j])
                if dist < data['dist_middle'].iloc[i]:    # 새로 구한 거리가 기존의 거리보다 짧을 때
                    data['dist_middle'].iloc[i] = dist       # 새로 구한 거리로 대체
                elif dist > data['dist_middle'].iloc[i]:  # 새로 구한 거리가 기존의 거리보다 길 때
                    continue                             # 다른 거리를 구한다.
                data['dist_middle'].iloc[i] = dist # <--기준점 거리(처음 구해진 거리)
    print("========== COMPLETED DINTANCE OF MIDDLE SCHOOL BETWEEN APT")
    print('---------- Done {:.2f} sec ----------'.format(time.time() - start))
    
    start = time.time()
    # < 8 > 고등학교와의 최단 거리
    for i in range(len(data)):
        for j in range(len(high)):
            if data['gu'].iloc[i] == high['gu'].iloc[j]:
                # 아파트 위치와 고등학교 위치의 거리를 구한다.
                dist = distance(data['apt_lat'].iloc[i],data['apt_lng'].iloc[i],high['lat'].iloc[j],high['lng'].iloc[j])
                if dist < data['dist_high'].iloc[i]:    # 새로 구한 거리가 기존의 거리보다 짧을 때
                    data['dist_high'].iloc[i] = dist       # 새로 구한 거리로 대체
                elif dist > data['dist_high'].iloc[i]:  # 새로 구한 거리가 기존의 거리보다 길 때
                    continue                             # 다른 거리를 구한다.
                data['dist_high'].iloc[i] = dist # <--기준점 거리(처음 구해진 거리)
    print("========== COMPLETED DINTANCE OF HIGH SCHOOL BETWEEN APT")
    print('---------- Done {:.2f} sec ----------'.format(time.time() - start))'''
    
    return data

In [514]:
duplicate_tr = calculate_distance(duplicate_tr)

data is ready to calculate
func of distance is working.....
each of distance columns are created


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


========== COMPLETED DINTANCE OF 'HAN'RIVER BETWEEN APT
---------- Done 39.11 sec ----------
========== COMPLETED DINTANCE OF SUBWAY BETWEEN APT
---------- Done 139.07 sec ----------
========== COMPLETED DINTANCE OF PARK BETWEEN APT
---------- Done 850.46 sec ----------
========== COMPLETED DINTANCE OF STARBUCKS BETWEEN APT
---------- Done 97.46 sec ----------
---------- Done All Function 1126.13 sec ----------


In [515]:
duplicate_tr

,gu,dong,addr_kr,apt,apt_lat,apt_lng,dist_river,dist_subway,dist_park,dist_starbucks
0,강남구,개포동,언주로 103,개포2차현대아파트(220),37.481344,127.052912,4.510525,0.805268,0.457460,0.792201
4,강남구,개포동,언주로 3,개포6차우성아파트1동~8동,37.476792,127.055926,4.926343,1.146684,0.594663,0.960569
11,강남구,개포동,개포로 307,개포우성3차,37.484631,127.056092,4.089917,0.346051,0.341695,0.546257
14,강남구,개포동,개포로109길 69,개포자이(12-2),37.496292,127.076127,2.607800,0.430890,0.235026,0.698292
18,강남구,개포동,삼성로 11,개포주공 3단지,37.484135,127.068092,3.932166,0.580087,0.130941,1.160478
...,...,...,...,...,...,...,...,...,...,...
799514,중구,황학동,난계로13길 36,로얄,37.567290,127.021571,2.242790,0.361657,0.236018,0.390768
800332,중랑구,면목동,사가정로42길 90,두산5,37.577959,127.081093,3.729996,0.690287,0.307793,0.632953
800651,중랑구,면목동,동일로101길 63,한신,37.588544,127.076312,4.854261,0.708908,0.069241,0.522734
801479,중랑구,상봉동,상봉로 131,상봉 듀오트리스,37.597407,127.092178,4.946715,0.239505,0.390980,0.059114


In [516]:
duplicate_te = calculate_distance(duplicate_te)

data is ready to calculate
func of distance is working.....
each of distance columns are created


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


========== COMPLETED DINTANCE OF 'HAN'RIVER BETWEEN APT
---------- Done 71.96 sec ----------
========== COMPLETED DINTANCE OF SUBWAY BETWEEN APT
---------- Done 266.20 sec ----------
========== COMPLETED DINTANCE OF PARK BETWEEN APT
---------- Done 1393.13 sec ----------
========== COMPLETED DINTANCE OF STARBUCKS BETWEEN APT
---------- Done 160.64 sec ----------
---------- Done All Function 1891.94 sec ----------


In [517]:
duplicate_te

,gu,dong,addr_kr,apt,apt_lat,apt_lng,dist_river,dist_subway,dist_park,dist_starbucks
0,강남구,개포동,언주로 103,개포2차현대아파트(220),37.482630,127.051104,4.410927,0.613785,0.330724,0.752602
1,강남구,개포동,언주로 3,개포6차우성아파트1동~8동,37.476658,127.056909,4.917798,1.145037,0.678788,1.048489
16,강남구,개포동,개포로 307,개포우성3차,37.484141,127.055247,4.163296,0.437509,0.293014,0.581530
33,강남구,개포동,개포로109길 69,개포자이(12-2),37.496094,127.076466,2.634732,0.394241,0.203622,0.733179
45,강남구,개포동,개포로 310,개포주공 1단지,37.476834,127.058210,4.870667,1.113965,0.795071,1.160523
...,...,...,...,...,...,...,...,...,...,...
254329,중랑구,신내동,망우로65길 16,조호그레이스2,37.601096,127.099935,4.993279,0.727651,0.282357,0.609354
254505,중랑구,중화동,망우로 231-8,미영리치타운103,37.595202,127.078436,5.315482,0.206879,0.336009,0.403342
254546,중랑구,중화동,봉화산로27길 30,한솔e(A동),37.603255,127.083636,5.849862,0.393186,0.306202,0.745784
254548,중랑구,중화동,봉화산로27길 30,한솔e(B동),37.603255,127.083636,5.849862,0.393186,0.306202,0.745784


In [518]:
train = pd.merge(train,duplicate_tr,how='left')
train.head(1)

,시군구,본번,부번,apt,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,...,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200606,1,60000,4,1988,...,28,134.0,84,6789,223.0,40,4.510525,0.805268,0.45746,0.792201


In [519]:
len(train)

803001

In [520]:
train.isna().sum()

시군구                       0
본번                        0
부번                        0
apt                       0
exclusive_area            0
transaction_ym            0
transaction_day           0
transaction_real_price    0
floor                     0
completion_year           0
addr_kr                   0
apt_lat                   0
apt_lng                   0
city                      0
gu                        0
dong                      0
code                      0
subway_cnt                0
park_cnt                  0
starbucks_cnt             0
academy_cnt               0
daycare_cnt               0
kindergarden_cnt          0
dist_river                0
dist_subway               0
dist_park                 0
dist_starbucks            0
dtype: int64

In [521]:
test = pd.merge(test,duplicate_te,how='left')
test.head(1)

,시군구,본번,부번,apt,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,...,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,201707,8,89900,1,1988,...,28,134.0,84,6789,223.0,40,4.410927,0.613785,0.330724,0.752602


In [522]:
len(test)

254674

In [523]:
test.isna().sum()

시군구                       0
본번                        0
부번                        0
apt                       0
exclusive_area            0
transaction_ym            0
transaction_day           0
transaction_real_price    0
floor                     0
completion_year           0
addr_kr                   0
apt_lat                   0
apt_lng                   0
city                      0
gu                        0
dong                      0
code                      0
subway_cnt                0
park_cnt                  0
starbucks_cnt             0
academy_cnt               0
daycare_cnt               0
kindergarden_cnt          0
dist_river                0
dist_subway               0
dist_park                 0
dist_starbucks            0
dtype: int64

In [525]:
park = pd.read_csv('data/ProcessedData/location/park_loc.csv', encoding='cp949')
kgarden = pd.read_csv('data/ProcessedData/location/kindergarden_loc.csv', encoding='cp949')
element = pd.read_csv('data/ProcessedData/location/element_loc.csv', encoding='cp949')
middle = pd.read_csv('data/ProcessedData/location/middle_loc.csv', encoding='cp949')

In [526]:
train.dist_river.describe()

count    803001.000000
mean          4.545035
std           3.651254
min           0.061781
25%           1.606633
50%           3.526729
75%           6.442639
max          15.393421
Name: dist_river, dtype: float64

In [527]:
test.dist_river.describe()

count    254674.000000
mean          4.457927
std           3.534845
min           0.057263
25%           1.634311
50%           3.491825
75%           6.280609
max          15.393421
Name: dist_river, dtype: float64

In [528]:
train.dist_subway.describe()

count    803001.000000
mean          0.599166
std           0.393376
min           0.005471
25%           0.343948
50%           0.515603
75%           0.730054
max           3.591522
Name: dist_subway, dtype: float64

In [529]:
test.dist_subway.describe()

count    254674.000000
mean          0.601530
std           0.395619
min           0.001985
25%           0.348713
50%           0.512824
75%           0.728968
max           3.814838
Name: dist_subway, dtype: float64

In [530]:
train.dist_park.describe()

count    803001.000000
mean          0.366921
std           0.381904
min           0.000075
25%           0.149253
50%           0.247015
75%           0.405473
max           2.397212
Name: dist_park, dtype: float64

In [533]:
train.dist_starbucks.describe()

count    803001.000000
mean          0.774059
std           0.563723
min           0.000912
25%           0.372087
50%           0.663673
75%           1.040966
max           5.075503
Name: dist_starbucks, dtype: float64

In [534]:
check = train.query("dist_starbucks >= 4")
check[['gu','dong','apt','addr_kr','apt_lat','apt_lng','dist_starbucks']]

,gu,dong,apt,addr_kr,apt_lat,apt_lng,dist_starbucks
46837,도봉구,도봉동,도봉산리베니움,도봉로181길 74-30,37.685758,127.043189,4.223722
46838,도봉구,도봉동,도봉산리베니움,도봉로181길 74-30,37.685758,127.043189,4.223722
46839,도봉구,도봉동,도봉산리베니움,도봉로181길 74-30,37.685758,127.043189,4.223722
46840,도봉구,도봉동,도봉산리베니움,도봉로181길 74-30,37.685758,127.043189,4.223722
46841,도봉구,도봉동,도봉산리베니움,도봉로181길 74-30,37.685758,127.043189,4.223722
...,...,...,...,...,...,...,...
740801,도봉구,도봉동,한신,마들로 859-19,37.686042,127.046765,4.320082
740802,도봉구,도봉동,한신,마들로 859-19,37.686042,127.046765,4.320082
740803,도봉구,도봉동,한신,마들로 859-19,37.686042,127.046765,4.320082
793869,용산구,한남동,아일랜드캐슬,대사관로7길 5,37.596741,126.989372,5.075503


In [535]:
check.dist_starbucks.unique()

array([4.2237217 , 4.22085104, 4.51545957, 4.24907088, 4.32398761,
       4.11849238, 4.39181151, 4.19666085, 4.3200823 , 5.07550285,
       5.03861875, 4.07871516, 4.01761364])

In [536]:
check.gu.value_counts()

도봉구    2192
용산구      32
Name: gu, dtype: int64

In [537]:
test.dist_starbucks.describe()

count    254674.000000
mean          0.768337
std           0.593292
min           0.000773
25%           0.354593
50%           0.659171
75%           1.012098
max          18.163381
Name: dist_starbucks, dtype: float64

In [538]:
check = test.query("dist_starbucks >= 4")
check[['gu','dong','apt','addr_kr','apt_lat','apt_lng','dist_starbucks']]

,gu,dong,apt,addr_kr,apt_lat,apt_lng,dist_starbucks
22337,관악구,남현동,흥화브라운빌,남현길 144,37.631611,127.043850,18.163381
22338,관악구,남현동,흥화브라운빌,남현길 144,37.631611,127.043850,18.163381
22339,관악구,남현동,흥화브라운빌,남현길 144,37.631611,127.043850,18.163381
25204,관악구,신림동,두영,문성로16나길 25,37.551736,126.920419,6.678607
25205,관악구,신림동,두영,문성로16나길 25,37.551736,126.920419,6.678607
...,...,...,...,...,...,...,...
203666,도봉구,도봉동,한신,마들로 859-19,37.686042,127.046765,4.320082
203667,도봉구,도봉동,한신,마들로 859-19,37.686042,127.046765,4.320082
203668,도봉구,도봉동,한신,마들로 859-19,37.686042,127.046765,4.320082
203669,도봉구,도봉동,한신,마들로 859-19,37.686042,127.046765,4.320082


In [539]:
check.dist_starbucks.unique()

array([18.16338092,  6.67860659,  4.7245301 ,  5.10474806,  4.00360447,
        4.21026092,  4.24734443,  4.51301622,  4.24446375,  4.27758941,
        4.08998968,  4.38319648,  4.19612121,  4.36011478, 14.93320104,
       10.08003287,  7.36184824,  6.42715295,  4.01761364,  4.2237217 ,
        4.22085104,  4.51545957,  4.32398761,  4.11849238,  4.19666085,
        4.3200823 , 16.19210057])

In [540]:
check.gu.value_counts()

도봉구     811
노원구      24
관악구      21
송파구      20
금천구      15
동대문구      4
성동구       3
서초구       2
서대문구      1
Name: gu, dtype: int64

# 각종 세권 평균매매가격 구하기 함수화

In [541]:
def influence_area(data):
    import numpy as np
    # 역세권
    super_SIA = int(np.round(data.query("dist_subway <= 0.2").transaction_real_price.values.mean(),0))
    SIA_1st = int(np.round(data.query("dist_subway > 0.2 & dist_subway <= 0.5").transaction_real_price.values.mean(),0))
    SIA_2nd = int(np.round(data.query("dist_subway > 0.5 & dist_subway <= 1.0").transaction_real_price.values.mean(),0))
    not_SIA = int(np.round(data.query("dist_subway > 1").transaction_real_price.values.mean(),0))
    
    data.loc[data.dist_subway <= 0.2, "StationIA(mean_price)"] = super_SIA
    data.loc[(data.dist_subway > 0.2)&(data.dist_subway <= 0.5), "StationIA(mean_price)"] = SIA_1st
    data.loc[(data.dist_subway > 0.5)&(data.dist_subway <= 1), "StationIA(mean_price)"] = SIA_2nd
    data.loc[data.dist_subway > 1, "StationIA(mean_price)"] = not_SIA

    # 스(타벅스)세권
    starbucks_1st = int(np.round(data.query("dist_starbucks <= 0.2").transaction_real_price.values.mean(),0))
    starbucks_2nd = int(np.round(data.query("dist_starbucks > 0.2 & dist_starbucks <= 0.5").transaction_real_price.values.mean(),0))
    starbucks_3rd = int(np.round(data.query("dist_starbucks > 0.5 & dist_starbucks <= 1.0").transaction_real_price.values.mean(),0))
    starbucks_4th = int(np.round(data.query("dist_starbucks > 1").transaction_real_price.values.mean(),0))
    
    data.loc[data.dist_starbucks <= 0.2, "StarbucksIA(mean_price)"] = starbucks_1st
    data.loc[(data.dist_starbucks > 0.2)&(data.dist_starbucks <= 0.5), "StarbucksIA(mean_price)"] = starbucks_2nd
    data.loc[(data.dist_starbucks > 0.5)&(data.dist_starbucks <= 1), "StarbucksIA(mean_price)"] = starbucks_3rd
    data.loc[data.dist_starbucks > 1, "StarbucksIA(mean_price)"] = starbucks_4th

    # 숲세권
    park_1st = int(np.round(data.query("dist_park <= 0.2").transaction_real_price.values.mean(),0))
    park_2nd = int(np.round(data.query("dist_park > 0.2 & dist_park <= 0.5").transaction_real_price.values.mean(),0))
    park_3rd = int(np.round(data.query("dist_park > 0.5 & dist_park <= 1.0").transaction_real_price.values.mean(),0))
    park_4th = int(np.round(data.query("dist_park > 1").transaction_real_price.values.mean(),0))

    data.loc[data.dist_park <= 0.2, "ParkIA(mean_price)"] = park_1st
    data.loc[(data.dist_park > 0.2)&(data.dist_park <= 0.5), "ParkIA(mean_price)"] = park_2nd
    data.loc[(data.dist_park > 0.5)&(data.dist_park <= 1), "ParkIA(mean_price)"] = park_3rd
    data.loc[data.dist_park > 1, "ParkIA(mean_price)"] = park_4th
    
    return data

In [542]:
train = influence_area(train)
train.head(1)

,시군구,본번,부번,apt,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,...,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price)
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200606,1,60000,4,1988,...,6789,223.0,40,4.510525,0.805268,0.45746,0.792201,44582.0,41268.0,46995.0


In [543]:
test = influence_area(test)
test.head(1)

,시군구,본번,부번,apt,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,...,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price)
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,201707,8,89900,1,1988,...,6789,223.0,40,4.410927,0.613785,0.330724,0.752602,68673.0,64696.0,73303.0


In [544]:
train['transaction_y'] = (train.transaction_ym / 100).astype(int)
test['transaction_y'] = (test.transaction_ym / 100).astype(int)

In [545]:
train.transaction_y

0         2006
1         2006
2         2006
3         2006
4         2006
          ... 
802996    2017
802997    2017
802998    2016
802999    2016
803000    2016
Name: transaction_y, Length: 803001, dtype: int64

In [546]:
test.transaction_y

0         2017
1         2017
2         2017
3         2017
4         2017
          ... 
254669    2020
254670    2020
254671    2020
254672    2020
254673    2019
Name: transaction_y, Length: 254674, dtype: int64

In [547]:
train.transaction_y.nunique()

12

In [548]:
test.transaction_y.nunique()

4

In [142]:
### 거래년월일 ###
'''train['transaction_m'] = train.transaction_ym - ((train.transaction_ym / 100).astype(int) * 100)
train.transaction_m

train['transaction_ymd'] = (train.transaction_ym * 100) + train.transaction_day
train.transaction_ymd

obj = ['transaction_ymd']
train[obj] = train[obj].apply(lambda x: x.astype('category').cat.codes)
train.transaction_ymd

train = train.assign(transaction_y = train.transaction_ymd.dt.year,
                    transaction_m = train.transaction_ymd.dt.month,
                    transaction_d = train.transaction_ymd.dt.day)
train.head(1)'''

In [455]:
### 지역별 총소득,개인소득(데이터가 2017까지만 존재하여 부적합함) ###
'''GRNI = pd.read_csv('data/행정구역 시도별 1인당 지역총소득,개인소득.csv', encoding='cp949')
GRNI

GRNI = GRNI.drop(0).set_index("행정구역(시도)별")
GRNI

GRNI = GRNI.rename(columns={'2017 p)':'2017','2017 p).1':'2017.1'})
GRNI

# 1인당 지역내 총소득
import numpy as np
train['GRNI'] = np.nan
for i in range(len(train)):
    if train['transaction_y'][i] == 2006:
        train['GRNI'][i] = GRNI['2006']
    elif train['transaction_y'][i] == 2007:
        train['GRNI'][i] = GRNI['2007']
    elif train['transaction_y'][i] == 2008:
        train['GRNI'][i] = GRNI['2008']
    elif train['transaction_y'][i] == 2009:
        train['GRNI'][i] = GRNI['2009']
    elif train['transaction_y'][i] == 2010:
        train['GRNI'][i] = GRNI['2010']
    elif train['transaction_y'][i] == 2011:
        train['GRNI'][i] = GRNI['2011']
    elif train['transaction_y'][i] == 2012:
        train['GRNI'][i] = GRNI['2012']
    elif train['transaction_y'][i] == 2013:
        train['GRNI'][i] = GRNI['2013']
    elif train['transaction_y'][i] == 2014:
        train['GRNI'][i] = GRNI['2014']
    elif train['transaction_y'][i] == 2015:
        train['GRNI'][i] = GRNI['2015']
    elif train['transaction_y'][i] == 2016:
        train['GRNI'][i] = GRNI['2016']
    elif train['transaction_y'][i] == 2017:
        train['GRNI'][i] = GRNI['2017']

# 1인당 지역내 개인소득
import numpy as np
train['GRPI'] = np.nan
for i in range(len(train)):
    if train['transaction_y'][i] == 2006:
        train['GRPI'][i] = GRNI['2006.1']
    elif train['transaction_y'][i] == 2007:
        train['GRPI'][i] = GRNI['2007.1']
    elif train['transaction_y'][i] == 2008:
        train['GRPI'][i] = GRNI['2008.1']
    elif train['transaction_y'][i] == 2009:
        train['GRPI'][i] = GRNI['2009.1']
    elif train['transaction_y'][i] == 2010:
        train['GRPI'][i] = GRNI['2010.1']
    elif train['transaction_y'][i] == 2011:
        train['GRPI'][i] = GRNI['2011.1']
    elif train['transaction_y'][i] == 2012:
        train['GRPI'][i] = GRNI['2012.1']
    elif train['transaction_y'][i] == 2013:
        train['GRPI'][i] = GRNI['2013.1']
    elif train['transaction_y'][i] == 2014:
        train['GRPI'][i] = GRNI['2014.1']
    elif train['transaction_y'][i] == 2015:
        train['GRPI'][i] = GRNI['2015.1']
    elif train['transaction_y'][i] == 2016:
        train['GRPI'][i] = GRNI['2016.1']
    elif train['transaction_y'][i] == 2017:
        train['GRPI'][i] = GRNI['2017.1']

# 1인당 지역내 총소득
import numpy as np
train['GRNI'] = np.nan
for i in range(len(train)):
    if train['transaction_y'][i] == 2018:
        train['GRNI'][i] = GRNI['2006']
    elif train['transaction_y'][i] == 2019:
        train['GRNI'][i] = GRNI['2007']
    elif train['transaction_y'][i] == 2020:
        train['GRNI'][i] = GRNI['2008']

# 1인당 지역내 개인소득
import numpy as np
train['GRPI'] = np.nan
for i in range(len(train)):
    if train['transaction_y'][i] == 2018:
        train['GRPI'][i] = GRNI['2006.1']
    elif train['transaction_y'][i] == 2019:
        train['GRPI'][i] = GRNI['2007.1']
    elif train['transaction_y'][i] == 2020:
        train['GRPI'][i] = GRNI['2008.1']'''

In [211]:
### 구별 주민등록 세대수 데이터 적용 ###
'''population = pd.read_excel('data/행정구역_시군구_별_주민등록세대수_20190107134842.xlsx')
population.head(50)

idx = list(range(2,26))

for i in range(len(idx)):
    population.at[idx[i], "행정구역(시군구)별(1)"] = '서울특별시'
population

population.columns = ['city','gu','households']
population.head(1)

population = population.query("city == '서울특별시'").drop(1)
population

population.shape

train = pd.merge(train,population,how='left')
train.head(1)

train.isna().sum()

train[train.households.isna()].head(1)

#상관계수 = 0.08'''

In [549]:
corr = train.corr()
corr_trsc = corr.transaction_real_price.sort_values(ascending=False).to_frame()
corr_trsc.style.background_gradient(cmap='magma')

,transaction_real_price
transaction_real_price,1.000000
exclusive_area,0.685985
starbucks_cnt,0.495723
academy_cnt,0.423830
subway_cnt,0.351667
park_cnt,0.351227
code,0.279919
StarbucksIA(mean_price),0.264333
transaction_ym,0.177885
transaction_y,0.177715


In [550]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 803001 entries, 0 to 803000
Data columns (total 31 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   시군구                      803001 non-null  object 
 1   본번                       803001 non-null  int64  
 2   부번                       803001 non-null  int64  
 3   apt                      803001 non-null  object 
 4   exclusive_area           803001 non-null  float64
 5   transaction_ym           803001 non-null  int64  
 6   transaction_day          803001 non-null  int64  
 7   transaction_real_price   803001 non-null  int64  
 8   floor                    803001 non-null  int64  
 9   completion_year          803001 non-null  int64  
 10  addr_kr                  803001 non-null  object 
 11  apt_lat                  803001 non-null  float64
 12  apt_lng                  803001 non-null  float64
 13  city                     803001 non-null  object 
 14  gu  

In [551]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 254674 entries, 0 to 254673
Data columns (total 31 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   시군구                      254674 non-null  object 
 1   본번                       254674 non-null  int64  
 2   부번                       254674 non-null  int64  
 3   apt                      254674 non-null  object 
 4   exclusive_area           254674 non-null  float64
 5   transaction_ym           254674 non-null  int64  
 6   transaction_day          254674 non-null  int64  
 7   transaction_real_price   254674 non-null  int64  
 8   floor                    254674 non-null  int64  
 9   completion_year          254674 non-null  int64  
 10  addr_kr                  254674 non-null  object 
 11  apt_lat                  254674 non-null  float64
 12  apt_lng                  254674 non-null  float64
 13  city                     254674 non-null  object 
 14  gu  

In [552]:
# 드랍할 컬럼 설정
obj = ['시군구','본번','부번','apt','transaction_ym','transaction_day','completion_year','addr_kr','city','gu','dong']
train = train.drop(obj, axis=1)
test = test.drop(obj, axis=1)

In [555]:
train

,exclusive_area,transaction_real_price,floor,apt_lat,apt_lng,code,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price),transaction_y
0,77.75,60000,4,37.481344,127.052912,1168010300,28,134.0,84,6789,223.0,40,4.510525,0.805268,0.457460,0.792201,44582.0,41268.0,46995.0,2006
1,77.75,72250,5,37.481344,127.052912,1168010300,28,134.0,84,6789,223.0,40,4.510525,0.805268,0.457460,0.792201,44582.0,41268.0,46995.0,2006
2,77.75,73500,8,37.481344,127.052912,1168010300,28,134.0,84,6789,223.0,40,4.510525,0.805268,0.457460,0.792201,44582.0,41268.0,46995.0,2006
3,77.75,76500,5,37.481344,127.052912,1168010300,28,134.0,84,6789,223.0,40,4.510525,0.805268,0.457460,0.792201,44582.0,41268.0,46995.0,2006
4,79.97,88000,3,37.476792,127.055926,1168010300,28,134.0,84,6789,223.0,40,4.926343,1.146684,0.594663,0.960569,32814.0,41268.0,39470.0,2006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802996,59.76,33000,22,37.597009,127.082071,1126010300,11,54.0,6,819,242.0,32,5.307876,0.270638,0.326711,0.386661,46057.0,52056.0,46995.0,2017
802997,84.03,40200,22,37.597009,127.082071,1126010300,11,54.0,6,819,242.0,32,5.307876,0.270638,0.326711,0.386661,46057.0,52056.0,46995.0,2017
802998,84.69,30000,4,37.606521,127.082165,1126010300,11,54.0,6,819,242.0,32,6.232857,0.510660,0.256695,0.908944,44582.0,41268.0,46995.0,2016
802999,67.44,25700,5,37.606521,127.082165,1126010300,11,54.0,6,819,242.0,32,6.232857,0.510660,0.256695,0.908944,44582.0,41268.0,46995.0,2016


In [556]:
test

,exclusive_area,transaction_real_price,floor,apt_lat,apt_lng,code,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price),transaction_y
0,77.75,89900,1,37.482630,127.051104,1168010300,28,134.0,84,6789,223.0,40,4.410927,0.613785,0.330724,0.752602,68673.0,64696.0,73303.0,2017
1,79.97,98000,1,37.476658,127.056909,1168010300,28,134.0,84,6789,223.0,40,4.917798,1.145037,0.678788,1.048489,50860.0,52210.0,62584.0,2017
2,54.98,79000,1,37.476658,127.056909,1168010300,28,134.0,84,6789,223.0,40,4.917798,1.145037,0.678788,1.048489,50860.0,52210.0,62584.0,2017
3,54.98,82000,3,37.476658,127.056909,1168010300,28,134.0,84,6789,223.0,40,4.917798,1.145037,0.678788,1.048489,50860.0,52210.0,62584.0,2017
4,54.98,78000,4,37.476658,127.056909,1168010300,28,134.0,84,6789,223.0,40,4.917798,1.145037,0.678788,1.048489,50860.0,52210.0,62584.0,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254669,58.87,28800,6,37.606521,127.082165,1126010300,11,54.0,6,819,242.0,32,6.232857,0.510660,0.256695,0.908944,68673.0,64696.0,73303.0,2020
254670,70.96,31200,4,37.606521,127.082165,1126010300,11,54.0,6,819,242.0,32,6.232857,0.510660,0.256695,0.908944,68673.0,64696.0,73303.0,2020
254671,66.24,28500,5,37.606521,127.082165,1126010300,11,54.0,6,819,242.0,32,6.232857,0.510660,0.256695,0.908944,68673.0,64696.0,73303.0,2020
254672,106.21,45400,6,37.606521,127.082165,1126010300,11,54.0,6,819,242.0,32,6.232857,0.510660,0.256695,0.908944,68673.0,64696.0,73303.0,2020


### 학습 데이터, 예측 데이터 나누기

In [557]:
feature = train.drop('transaction_real_price', axis=1)     # 학습 데이터
target = train.transaction_real_price.values               # 예측할 타겟
answer = test.transaction_real_price.values.copy()         # 타겟 실제값
pred = test.drop('transaction_real_price', axis=1)         # 예측 데이터

### 모델 알고리즘 설정

In [562]:
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

lgb = LGBMRegressor(random_state=0, max_depth=-1, min_child_samples=0,min_data=0,
                               min_child_weight=1, min_data_in_bin=1, min_split_gain=0.0,
                               n_estimators=600, n_jobs=-1, num_class=1,num_leaves=10)

xgb = XGBRegressor(colsample_bytree=0.4603, gamma=0.0468,learning_rate=0.05, max_depth=3, 
                         min_child_weight=1.7817, n_estimators=2200,reg_alpha=0.4640, reg_lambda=0.8571,
                         subsample=0.5213, silent=1,random_state =7, nthread = -1,n_jobs=-1)

gb = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=4, max_features='sqrt',
                              min_samples_leaf=15, min_samples_split=10, loss='huber', random_state=5)

cat = CatBoostRegressor(iterations=5000, learning_rate=0.2, depth=5, loss_function='RMSE', verbose=True)

### 데이터 학습

In [568]:
model_lgb = lgb.fit(feature, target)

In [612]:
model_xgb = xgb.fit(feature, target)

In [620]:
model_gb = gb.fit(feature, target)

In [578]:
model_cat = cat.fit(feature, target)

0:	learn: 27733.1669056	total: 136ms	remaining: 11m 18s
1:	learn: 24717.7903857	total: 200ms	remaining: 8m 19s
2:	learn: 22469.5782023	total: 260ms	remaining: 7m 12s
3:	learn: 20621.4694805	total: 325ms	remaining: 6m 45s
4:	learn: 19316.3165814	total: 393ms	remaining: 6m 32s
5:	learn: 18327.3794528	total: 477ms	remaining: 6m 37s
6:	learn: 17503.2017782	total: 564ms	remaining: 6m 42s
7:	learn: 16869.3101132	total: 630ms	remaining: 6m 33s
8:	learn: 16350.4781233	total: 699ms	remaining: 6m 27s
9:	learn: 15973.9268018	total: 772ms	remaining: 6m 25s
10:	learn: 15617.0825521	total: 834ms	remaining: 6m 18s
11:	learn: 15289.4282459	total: 905ms	remaining: 6m 16s
12:	learn: 15030.2934927	total: 972ms	remaining: 6m 13s
13:	learn: 14805.3680579	total: 1.04s	remaining: 6m 9s
14:	learn: 14599.4487268	total: 1.1s	remaining: 6m 4s
15:	learn: 14399.8060105	total: 1.17s	remaining: 6m 4s
16:	learn: 14204.6638457	total: 1.24s	remaining: 6m 3s
17:	learn: 14021.8652487	total: 1.3s	remaining: 5m 59s
18:	lea

### 전개

In [719]:
pred.head()

,exclusive_area,floor,apt_lat,apt_lng,code,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price),transaction_y,transaction_real_price
0,77.75,1,37.482630,127.051104,1168010300,28,134.0,84,6789,223.0,40,4.410927,0.613785,0.330724,0.752602,68673.0,64696.0,73303.0,2017,88676.0
1,79.97,1,37.476658,127.056909,1168010300,28,134.0,84,6789,223.0,40,4.917798,1.145037,0.678788,1.048489,50860.0,52210.0,62584.0,2017,103808.0
2,54.98,1,37.476658,127.056909,1168010300,28,134.0,84,6789,223.0,40,4.917798,1.145037,0.678788,1.048489,50860.0,52210.0,62584.0,2017,80046.0
3,54.98,3,37.476658,127.056909,1168010300,28,134.0,84,6789,223.0,40,4.917798,1.145037,0.678788,1.048489,50860.0,52210.0,62584.0,2017,81899.0
4,54.98,4,37.476658,127.056909,1168010300,28,134.0,84,6789,223.0,40,4.917798,1.145037,0.678788,1.048489,50860.0,52210.0,62584.0,2017,81950.0


In [720]:
pred = pred.drop("transaction_real_price", axis=1)
pred.head()

,exclusive_area,floor,apt_lat,apt_lng,code,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price),transaction_y
0,77.75,1,37.482630,127.051104,1168010300,28,134.0,84,6789,223.0,40,4.410927,0.613785,0.330724,0.752602,68673.0,64696.0,73303.0,2017
1,79.97,1,37.476658,127.056909,1168010300,28,134.0,84,6789,223.0,40,4.917798,1.145037,0.678788,1.048489,50860.0,52210.0,62584.0,2017
2,54.98,1,37.476658,127.056909,1168010300,28,134.0,84,6789,223.0,40,4.917798,1.145037,0.678788,1.048489,50860.0,52210.0,62584.0,2017
3,54.98,3,37.476658,127.056909,1168010300,28,134.0,84,6789,223.0,40,4.917798,1.145037,0.678788,1.048489,50860.0,52210.0,62584.0,2017
4,54.98,4,37.476658,127.056909,1168010300,28,134.0,84,6789,223.0,40,4.917798,1.145037,0.678788,1.048489,50860.0,52210.0,62584.0,2017


## LightGBM Regressor

In [714]:
## LightGBM Regressor
pred['transaction_real_price'] = np.round(model_lgb.predict(pred),0)  # 예측
error_val = answer - pred.transaction_real_price          # 오차값
print("평균 오차값 :", abs(error_val).mean())                # 평균 오차값
error_rat = error_val / answer * 100                      # 오차율
print("평균 오차율 :", abs(error_rat).mean())                # 평균 오차율
print(abs(error_rat).describe())                          # 오차율 요약 통계량

오차값들의 평균 : 18097.159525511044
오차율들의 평균 : 22.28141171503874
count    254674.000000
mean         22.281412
std          14.780407
min           0.000000
25%          10.170139
50%          21.053641
75%          32.429798
max         259.093333
Name: transaction_real_price, dtype: float64


In [673]:
100 - abs(error_rat).mean()  # 평균예측률

77.71858828496126

In [674]:
len(error_rat[error_rat <= 10.170139])  # 오차율 최소값부터 1분위수까지 개수

87578

### 오차율 최대값 데이터

In [675]:
error_rat[error_rat <= -259]  # 오차율 최대값

103368   -259.093333
Name: transaction_real_price, dtype: float64

In [676]:
pred.query("index == 103368")

,exclusive_area,floor,apt_lat,apt_lng,code,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price),transaction_y,transaction_real_price
103368,60.63,3,37.538131,127.005579,1117013100,17,42.0,20,474,0.0,17,1.01737,0.378523,0.732259,0.410853,74841.0,84016.0,62584.0,2017,53864.0


In [677]:
accurate = pd.read_csv('data/ProcessedData/test_final.csv', encoding='cp949')
accurate.query("index == 103368")

,시군구,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,위도,경도
103368,서울특별시 용산구 한남동,723,3,한남하우스,60.63,201708,22,"15,000",3,1970,한남대로28가길 5,37.538131,127.005579


In [717]:
accurate.query("도로명 == '한남대로28가길 5'")

,시군구,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,위도,경도
103366,서울특별시 용산구 한남동,723,3,한남하우스,60.63,201707,8,"23,000",3,1970,한남대로28가길 5,37.538131,127.005579
103367,서울특별시 용산구 한남동,723,3,한남하우스,60.63,201708,2,"23,000",2,1970,한남대로28가길 5,37.538131,127.005579
103368,서울특별시 용산구 한남동,723,3,한남하우스,60.63,201708,22,"15,000",3,1970,한남대로28가길 5,37.538131,127.005579
161310,서울특별시 용산구 한남동,723,3,한남하우스,60.63,201809,27,"25,500",3,1970,한남대로28가길 5,37.538174,127.005607
246844,서울특별시 용산구 한남동,723,3,한남하우스,68.55,201905,9,"28,000",6,1970,한남대로28가길 5,37.538174,127.005607


In [718]:
pred.query("index in [103366,103367,103368,161310,246844]")

,exclusive_area,floor,apt_lat,apt_lng,code,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price),transaction_y,transaction_real_price
103366,60.63,3,37.538131,127.005579,1117013100,17,42.0,20,474,0.0,17,1.017370,0.378523,0.732259,0.410853,74841.0,84016.0,62584.0,2017,53864.0
103367,60.63,2,37.538131,127.005579,1117013100,17,42.0,20,474,0.0,17,1.017370,0.378523,0.732259,0.410853,74841.0,84016.0,62584.0,2017,52189.0
103368,60.63,3,37.538131,127.005579,1117013100,17,42.0,20,474,0.0,17,1.017370,0.378523,0.732259,0.410853,74841.0,84016.0,62584.0,2017,53864.0
161310,60.63,3,37.538174,127.005607,1117013100,17,42.0,20,474,0.0,17,1.021127,0.378638,0.732915,0.414533,74841.0,84016.0,62584.0,2018,53864.0
246844,68.55,6,37.538174,127.005607,1117013100,17,42.0,20,474,0.0,17,1.021127,0.378638,0.732915,0.414533,74841.0,84016.0,62584.0,2019,56290.0


### 오차율 최소값 데이터

In [638]:
error_rat[error_rat == 0]  # 오차율 0으로 정확하게 예측한 데이터의 인덱스번호

8447      0.0
14307     0.0
37595     0.0
38386     0.0
100443    0.0
107188    0.0
166603    0.0
189404    0.0
253821    0.0
Name: transaction_real_price, dtype: float64

In [608]:
pred.query("index in [8447,14307,37595,38386,100443,107188,166603,189404,253821]")

,exclusive_area,floor,apt_lat,apt_lng,code,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price),transaction_y,transaction_real_price
8447,61.120,1,37.541569,127.147426,1174010500,10,74.0,13,1914,272.0,39,2.494030,0.592798,0.198753,0.725845,68673.0,64696.0,72375.0,2017,37500.0
14307,114.850,8,37.617309,127.012633,1130510100,3,45.0,5,654,167.0,24,7.744292,1.570339,0.409209,0.755985,50860.0,64696.0,73303.0,2017,48100.0
37595,58.010,10,37.647499,127.059820,1135010500,15,2.0,9,2241,457.0,70,11.094552,0.510751,0.660955,0.218405,68673.0,84016.0,62584.0,2017,31500.0
38386,59.390,9,37.668468,127.051987,1135010500,15,2.0,9,2241,457.0,70,13.287297,0.636037,0.640438,0.947279,68673.0,64696.0,62584.0,2017,30000.0
100443,84.908,9,37.511707,126.903562,1156010100,18,42.0,30,1401,266.0,41,2.000406,0.553993,0.181400,0.613013,68673.0,64696.0,72375.0,2017,54800.0
107188,60.000,6,37.575200,126.960415,1111018700,15,34.0,33,822,78.0,17,4.476879,0.245261,0.258777,0.405566,74841.0,84016.0,73303.0,2017,44800.0
166603,33.180,5,37.610867,127.095652,1126010600,11,54.0,6,819,242.0,32,6.121739,0.703275,0.071620,0.607916,68673.0,64696.0,72375.0,2018,19300.0
189404,59.950,1,37.502866,126.891947,1153010200,13,47.0,12,1281,334.0,38,3.232052,0.654009,0.180938,0.488599,68673.0,84016.0,72375.0,2019,33500.0
253821,84.600,8,37.603570,127.101205,1126010600,11,54.0,6,819,242.0,32,5.174841,0.679532,0.186690,0.529717,68673.0,64696.0,72375.0,2019,39000.0


In [610]:
accurate = pd.read_csv('data/ProcessedData/test_final.csv', encoding='cp949')
accurate.query("index in [8447,14307,37595,38386,100443,107188,166603,189404,253821]")

,시군구,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,위도,경도
8447,서울특별시 강동구 길동,160,0,신동아,61.120,201710,24,"37,500",1,1983,천중로 264,37.541569,127.147426
14307,서울특별시 강북구 미아동,1353,0,에스케이북한산시티,114.850,201708,16,"48,100",8,2004,솔샘로 174,37.617309,127.012633
37595,서울특별시 노원구 상계동,765,0,상계주공1(고층),58.010,201706,6,"31,500",10,1988,덕릉로 459-21,37.647499,127.059820
38386,서울특별시 노원구 상계동,624,0,상계주공16(고층),59.390,201706,5,"30,000",9,1988,동일로227길 86,37.668468,127.051987
100443,서울특별시 영등포구 영등포동,647,0,영등포푸르지오,84.908,201706,27,"54,800",9,2002,도신로29길 28,37.511707,126.903562
107188,서울특별시 종로구 무악동,82,0,현대,60.000,201705,27,"44,800",6,2000,통일로 246-20,37.575200,126.960415
166603,서울특별시 중랑구 신내동,657,0,신내9진흥,33.180,201806,2,"19,300",5,1996,신내로 127,37.610867,127.095652
189404,서울특별시 구로구 구로동,43,4,다솜금호,59.950,201910,29,"33,500",1,1995,새말로16길 70,37.502866,126.891947
253821,서울특별시 중랑구 신내동,787,0,석탑,84.600,201907,20,"39,000",8,1997,용마산로125길 92,37.603570,127.101205


## XGB Regressor

In [721]:
## XGB Regressor
pred['transaction_real_price'] = np.round(model_xgb.predict(pred),0)  # 예측
error_val = answer - pred.transaction_real_price          # 오차값
print("평균 오차값 :", abs(error_val).mean())                # 평균 오차값
error_rat = error_val / answer * 100                      # 오차율
print("평균 오차율 :", abs(error_rat).mean())                # 평균 오차율
print(abs(error_rat).describe())                          # 오차율 요약 통계량

오차값들의 평균 : 18178.486328125
오차율들의 평균 : 22.74077606201172
count    254674.000000
mean         22.740776
std          15.055151
min           0.000000
25%          10.560109
50%          21.539704
75%          32.845760
max         298.773346
Name: transaction_real_price, dtype: float64


In [722]:
100 - abs(error_rat).mean()  # 평균예측률

77.25922393798828

In [688]:
len(error_rat[error_rat <= 10.560109])  # 오차율 최소값부터 1분위수까지 개수

89653

### 오차율 최대값 데이터

In [689]:
error_rat[error_rat <= -298]  # 오차율 최대값

103368   -298.773346
Name: transaction_real_price, dtype: float32

In [690]:
pred.query("index == 103368")

,exclusive_area,floor,apt_lat,apt_lng,code,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price),transaction_y,transaction_real_price
103368,60.63,3,37.538131,127.005579,1117013100,17,42.0,20,474,0.0,17,1.01737,0.378523,0.732259,0.410853,74841.0,84016.0,62584.0,2017,59816.0


In [691]:
accurate = pd.read_csv('data/ProcessedData/test_final.csv', encoding='cp949')
accurate.query("index == 103368")

,시군구,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,위도,경도
103368,서울특별시 용산구 한남동,723,3,한남하우스,60.63,201708,22,"15,000",3,1970,한남대로28가길 5,37.538131,127.005579


In [723]:
accurate.query("도로명 == '한남대로28가길 5'")

,시군구,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,위도,경도
103366,서울특별시 용산구 한남동,723,3,한남하우스,60.63,201707,8,"23,000",3,1970,한남대로28가길 5,37.538131,127.005579
103367,서울특별시 용산구 한남동,723,3,한남하우스,60.63,201708,2,"23,000",2,1970,한남대로28가길 5,37.538131,127.005579
103368,서울특별시 용산구 한남동,723,3,한남하우스,60.63,201708,22,"15,000",3,1970,한남대로28가길 5,37.538131,127.005579
161310,서울특별시 용산구 한남동,723,3,한남하우스,60.63,201809,27,"25,500",3,1970,한남대로28가길 5,37.538174,127.005607
246844,서울특별시 용산구 한남동,723,3,한남하우스,68.55,201905,9,"28,000",6,1970,한남대로28가길 5,37.538174,127.005607


In [724]:
pred.query("index in [103366,103367,103368,161310,246844]")

,exclusive_area,floor,apt_lat,apt_lng,code,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price),transaction_y,transaction_real_price
103366,60.63,3,37.538131,127.005579,1117013100,17,42.0,20,474,0.0,17,1.017370,0.378523,0.732259,0.410853,74841.0,84016.0,62584.0,2017,59816.0
103367,60.63,2,37.538131,127.005579,1117013100,17,42.0,20,474,0.0,17,1.017370,0.378523,0.732259,0.410853,74841.0,84016.0,62584.0,2017,58210.0
103368,60.63,3,37.538131,127.005579,1117013100,17,42.0,20,474,0.0,17,1.017370,0.378523,0.732259,0.410853,74841.0,84016.0,62584.0,2017,59816.0
161310,60.63,3,37.538174,127.005607,1117013100,17,42.0,20,474,0.0,17,1.021127,0.378638,0.732915,0.414533,74841.0,84016.0,62584.0,2018,59133.0
246844,68.55,6,37.538174,127.005607,1117013100,17,42.0,20,474,0.0,17,1.021127,0.378638,0.732915,0.414533,74841.0,84016.0,62584.0,2019,60143.0


### 오차율 최소값 데이터

In [692]:
error_rat[error_rat == 0]  # 오차율 0으로 정확하게 예측한 데이터의 인덱스번호

8657      0.0
9225      0.0
29664     0.0
42695     0.0
43362     0.0
43441     0.0
44099     0.0
103608    0.0
150922    0.0
245254    0.0
Name: transaction_real_price, dtype: float32

In [618]:
pred.query("index in [8657,9225,29664,42695,43362,43441,44099,103608,150922,245254]")

,exclusive_area,floor,apt_lat,apt_lng,code,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price),transaction_y,transaction_real_price
8657,72.410,11,37.537986,127.136177,1174010500,10,74.0,13,1914,272.0,39,1.661004,0.338046,0.110719,0.315846,74841.0,84016.0,72375.0,2017,40000.0
9225,59.790,20,37.529348,127.142399,1174010600,10,74.0,13,1914,272.0,39,2.658391,0.569473,0.245168,0.690157,68673.0,64696.0,73303.0,2017,42000.0
29664,59.900,15,37.500929,126.865163,1153010600,13,47.0,12,1281,334.0,38,4.999153,0.679635,0.456581,1.731663,68673.0,52210.0,73303.0,2018,31500.0
42695,115.260,1,37.646777,127.072764,1135010600,15,2.0,9,2241,457.0,70,10.592268,0.783831,1.787184,0.501288,68673.0,64696.0,44028.0,2017,59300.0
43362,44.520,15,37.657692,127.078735,1135010600,15,2.0,9,2241,457.0,70,11.460692,0.576238,1.600328,0.835960,68673.0,64696.0,44028.0,2017,22000.0
43441,44.520,15,37.657692,127.078735,1135010600,15,2.0,9,2241,457.0,70,11.460692,0.576238,1.600328,0.835960,68673.0,64696.0,44028.0,2018,22000.0
44099,39.910,6,37.641576,127.063504,1135010600,15,2.0,9,2241,457.0,70,10.458989,0.341707,1.035479,0.741041,74841.0,64696.0,44028.0,2017,22100.0
103608,59.595,13,37.618871,126.913695,1138010400,13,98.0,10,1563,283.0,50,6.526291,0.644271,0.182041,0.561628,68673.0,64696.0,72375.0,2017,28500.0
150922,59.590,14,37.604846,127.017805,1129013300,9,0.0,13,1392,0.0,55,6.289758,0.658197,0.249939,1.039636,68673.0,52210.0,73303.0,2018,35800.0
245254,70.010,6,37.526037,126.909945,1156010800,18,42.0,30,1401,266.0,41,1.162617,0.565589,0.118995,0.441652,68673.0,84016.0,72375.0,2019,39200.0


In [619]:
accurate = pd.read_csv('data/ProcessedData/test_final.csv', encoding='cp949')
accurate.query("index in [8657,9225,29664,42695,43362,43441,44099,103608,150922,245254]")

,시군구,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,위도,경도
8657,서울특별시 강동구 길동,384,1,한빛골드빌1차,72.410,201709,1,"40,000",11,2001,성안로 176,37.537986,127.136177
9225,서울특별시 강동구 둔촌동,545,0,둔촌하이츠,59.790,201705,15,"42,000",20,1998,명일로 113,37.529348,127.142399
29664,서울특별시 구로구 고척동,327,0,동아한신,59.900,201801,3,"31,500",15,2000,중앙로6길 17,37.500929,126.865163
42695,서울특별시 노원구 중계동,361,1,롯데우성아파트,115.260,201706,20,"59,300",1,1993,노원로22길 34,37.646777,127.072764
43362,서울특별시 노원구 중계동,369,0,주공2,44.520,201707,28,"22,000",15,1992,한글비석로14길 8,37.657692,127.078735
43441,서울특별시 노원구 중계동,369,0,주공2,44.520,201803,10,"22,000",15,1992,한글비석로14길 8,37.657692,127.078735
44099,서울특별시 노원구 중계동,502,1,중계그린1단지,39.910,201706,3,"22,100",6,1990,동일로207길 17,37.641576,127.063504
103608,서울특별시 은평구 갈현동,538,0,미미,59.595,201705,12,"28,500",13,1997,갈현로29길 11,37.618871,126.913695
150922,서울특별시 성북구 정릉동,1015,0,경남,59.590,201806,30,"35,800",14,1999,정릉로 305,37.604846,127.017805
245254,서울특별시 영등포구 영등포동7가,94,41,삼성,70.010,201909,26,"39,200",6,2003,버드나루로14가길 8,37.526037,126.909945


## GradientBoosting Regressor

In [706]:
## GradientBoosting Regressor
pred['transaction_real_price'] = np.round(model_gb.predict(pred),0)   # 예측
error_val = answer - pred.transaction_real_price          # 오차값
print("평균 오차값 :", abs(error_val).mean())                # 평균 오차값
error_rat = error_val / answer * 100                      # 오차율
print("평균 오차율 :", abs(error_rat).mean())                # 평균 오차율
print(abs(error_rat).describe())                          # 오차율 요약 통계량

오차값들의 평균 : 17953.87066995453
오차율들의 평균 : 21.750332533308676
count    254674.000000
mean         21.750333
std          14.099170
min           0.000000
25%          10.022548
50%          20.866905
75%          31.677143
max         301.333333
Name: transaction_real_price, dtype: float64


In [707]:
100 - abs(error_rat).mean()  # 평균예측률

78.24966746669132

In [650]:
len(error_rat[error_rat <= 10.022548])  # 오차율 최소값부터 1분위수까지 개수

81782

### 오차율 최대값 데이터

In [710]:
error_rat[error_rat <= -301]  # 오차율 최대값

103368   -301.333333
Name: transaction_real_price, dtype: float64

In [711]:
pred.query("index == 103368")

,exclusive_area,floor,apt_lat,apt_lng,code,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price),transaction_y,transaction_real_price
103368,60.63,3,37.538131,127.005579,1117013100,17,42.0,20,474,0.0,17,1.01737,0.378523,0.732259,0.410853,74841.0,84016.0,62584.0,2017,60200.0


In [696]:
accurate = pd.read_csv('data/ProcessedData/test_final.csv', encoding='cp949')
accurate.query("index == 103368")

,시군구,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,위도,경도
103368,서울특별시 용산구 한남동,723,3,한남하우스,60.63,201708,22,"15,000",3,1970,한남대로28가길 5,37.538131,127.005579


In [708]:
accurate.query("도로명 == '한남대로28가길 5'")

,시군구,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,위도,경도
103366,서울특별시 용산구 한남동,723,3,한남하우스,60.63,201707,8,"23,000",3,1970,한남대로28가길 5,37.538131,127.005579
103367,서울특별시 용산구 한남동,723,3,한남하우스,60.63,201708,2,"23,000",2,1970,한남대로28가길 5,37.538131,127.005579
103368,서울특별시 용산구 한남동,723,3,한남하우스,60.63,201708,22,"15,000",3,1970,한남대로28가길 5,37.538131,127.005579
161310,서울특별시 용산구 한남동,723,3,한남하우스,60.63,201809,27,"25,500",3,1970,한남대로28가길 5,37.538174,127.005607
246844,서울특별시 용산구 한남동,723,3,한남하우스,68.55,201905,9,"28,000",6,1970,한남대로28가길 5,37.538174,127.005607


In [709]:
pred.query("index in [103366,103367,103368,161310,246844]")

,exclusive_area,floor,apt_lat,apt_lng,code,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price),transaction_y,transaction_real_price
103366,60.63,3,37.538131,127.005579,1117013100,17,42.0,20,474,0.0,17,1.017370,0.378523,0.732259,0.410853,74841.0,84016.0,62584.0,2017,60200.0
103367,60.63,2,37.538131,127.005579,1117013100,17,42.0,20,474,0.0,17,1.017370,0.378523,0.732259,0.410853,74841.0,84016.0,62584.0,2017,59148.0
103368,60.63,3,37.538131,127.005579,1117013100,17,42.0,20,474,0.0,17,1.017370,0.378523,0.732259,0.410853,74841.0,84016.0,62584.0,2017,60200.0
161310,60.63,3,37.538174,127.005607,1117013100,17,42.0,20,474,0.0,17,1.021127,0.378638,0.732915,0.414533,74841.0,84016.0,62584.0,2018,60118.0
246844,68.55,6,37.538174,127.005607,1117013100,17,42.0,20,474,0.0,17,1.021127,0.378638,0.732915,0.414533,74841.0,84016.0,62584.0,2019,62768.0


### 오차율 최소값 데이터

In [656]:
error_rat[error_rat == 0]  # 오차율 0으로 정확하게 예측한 데이터의 인덱스번호

19006     0.0
19441     0.0
36491     0.0
46099     0.0
47155     0.0
92582     0.0
145274    0.0
153417    0.0
199546    0.0
203790    0.0
214609    0.0
218603    0.0
Name: transaction_real_price, dtype: float64

In [657]:
pred.query("index in [19006,19441,36491,46099,47155,92582,145274,153417,199546,203790,214609,218603]")

,exclusive_area,floor,apt_lat,apt_lng,code,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price),transaction_y,transaction_real_price
19006,59.340,12,37.570790,126.806833,1150010900,19,139.0,17,2121,874.0,46,1.991415,0.188212,0.040776,0.933440,80385.0,64696.0,72375.0,2018,37500.0
19441,84.930,2,37.575003,126.809625,1150010900,19,139.0,17,2121,874.0,46,1.463302,0.386289,0.256574,0.597844,74841.0,64696.0,73303.0,2018,43500.0
36491,13.620,8,37.617535,127.074186,1135010300,15,2.0,9,2241,457.0,70,7.642376,0.096151,0.407829,0.996641,80385.0,64696.0,73303.0,2018,13200.0
46099,84.860,11,37.679383,127.044404,1132010800,7,71.0,1,1005,255.0,31,14.436551,0.106675,0.201427,3.551486,80385.0,52210.0,73303.0,2018,33500.0
47155,126.890,8,37.660163,127.027851,1132010600,7,71.0,1,1005,255.0,31,12.284548,1.417735,0.114297,1.451707,50860.0,52210.0,72375.0,2017,46300.0
92582,59.648,15,37.520399,126.838429,1147010300,6,96.0,11,3186,316.0,50,5.067429,1.277819,0.183565,1.153380,50860.0,52210.0,72375.0,2018,30200.0
145274,64.260,7,37.597194,126.945085,1141011800,7,52.0,20,1035,154.0,26,6.182897,0.911614,0.262410,0.955418,68673.0,64696.0,73303.0,2019,29500.0
153417,59.300,3,37.507336,127.121542,1171011100,19,159.0,28,3153,409.0,53,2.661926,0.438901,0.259852,0.125949,74841.0,93476.0,73303.0,2018,44000.0
199546,49.770,5,37.631814,127.050545,1135010200,15,2.0,9,2241,457.0,70,9.231683,0.746079,0.701854,1.166503,68673.0,52210.0,62584.0,2019,24500.0
203790,53.700,10,37.662866,127.027846,1132010600,7,71.0,1,1005,255.0,31,12.584824,1.535109,0.066468,1.728589,50860.0,52210.0,72375.0,2019,20600.0


In [658]:
accurate = pd.read_csv('data/ProcessedData/test_final.csv', encoding='cp949')
accurate.query("index in [19006,19441,36491,46099,47155,92582,145274,153417,199546,203790,214609,218603]")

,시군구,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,위도,경도
19006,서울특별시 강서구 방화동,852,0,방화12도시개발공사,59.340,201802,8,"37,500",12,1994,양천로6길 28,37.570790,126.806833
19441,서울특별시 강서구 방화동,878,0,월드메르디앙 101동,84.930,201801,23,"43,500",2,2004,양천로1길 117,37.575003,126.809625
36491,서울특별시 노원구 공릉동,683,14,한일휴니스빌,13.620,201801,12,"13,200",8,2013,화랑로 421,37.617535,127.074186
46099,서울특별시 도봉구 도봉동,595,0,럭키,84.860,201803,1,"33,500",11,1992,도봉로169길 31,37.679383,127.044404
47155,서울특별시 도봉구 방학동,271,1,신동아아파트1,126.890,201708,14,"46,300",8,1986,방학로 193,37.660163,127.027851
92582,서울특별시 양천구 신월동,1017,0,대방샤인힐,59.648,201801,31,"30,200",15,2003,신월로9길 37,37.520399,126.838429
145274,서울특별시 서대문구 홍은동,22,0,유원,64.260,201902,11,"29,500",7,1987,홍은중앙로1길 33,37.597194,126.945085
153417,서울특별시 송파구 방이동,223,13,AQUADCASAS,59.300,201810,6,"44,000",3,2004,오금로31길 15-5,37.507336,127.121542
199546,서울특별시 노원구 월계동,780,0,청백3,49.770,201907,5,"24,500",5,1998,월계로45가길 89,37.631814,127.050545
203790,서울특별시 도봉구 방학동,444,31,대원그린아파트,53.700,201907,23,"20,600",10,1987,방학로 212-11,37.662866,127.027846


## CatBoost Regressor

In [702]:
## CatBoost Regressor
pred['transaction_real_price'] = np.round(model_cat.predict(pred),0)  # 예측
error_val = answer - pred.transaction_real_price          # 오차값
print("평균 오차값 :", abs(error_val).mean())                # 평균 오차값
error_rat = error_val / answer * 100                      # 오차율
print("평균 오차율 :", abs(error_rat).mean())                # 평균 오차율
print(abs(error_rat).describe())                          # 오차율 요약 통계량

오차값들의 평균 : 17709.07223744866
오차율들의 평균 : 21.989233263848103
count    254674.000000
mean         21.989233
std          13.333119
min           0.000000
25%          10.969052
50%          21.753363
75%          31.623711
max         249.023529
Name: transaction_real_price, dtype: float64


In [662]:
100 - abs(error_rat).mean()  # 평균예측률

78.0107667361519

In [663]:
len(error_rat[error_rat <= 10.969052])  # 오차율 최소값부터 1분위수까지 개수

73805

### 오차율 최대값 데이터

In [664]:
error_rat[error_rat <= -249]  # 오차율 최대값

57984   -249.023529
Name: transaction_real_price, dtype: float64

In [668]:
pred.query("index == 57984")

,exclusive_area,floor,apt_lat,apt_lng,code,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price),transaction_y,transaction_real_price
57984,84.46,4,37.493668,126.925918,1159010900,17,52.0,8,1374,234.0,34,2.181132,0.702318,0.376054,0.336717,68673.0,84016.0,73303.0,2017,59334.0


In [669]:
accurate = pd.read_csv('data/ProcessedData/test_final.csv', encoding='cp949')
accurate.query("index == 57984")

,시군구,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,위도,경도
57984,서울특별시 동작구 신대방동,344,179,뉴홍현,84.46,201705,24,"17,000",4,2002,보라매로7가길 9,37.493668,126.925918


In [700]:
accurate.query("도로명 == '보라매로7가길 9'")

,시군구,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,위도,경도
57983,서울특별시 동작구 신대방동,344,179,뉴홍현,84.46,201705,10,"18,111",2,2002,보라매로7가길 9,37.493668,126.925918
57984,서울특별시 동작구 신대방동,344,179,뉴홍현,84.46,201705,24,"17,000",4,2002,보라매로7가길 9,37.493668,126.925918
57985,서울특별시 동작구 신대방동,344,179,뉴홍현,84.46,201706,16,"34,000",4,2002,보라매로7가길 9,37.493668,126.925918
57986,서울특별시 동작구 신대방동,344,179,뉴홍현,84.46,201803,31,"38,500",5,2002,보라매로7가길 9,37.493668,126.925918
141923,서울특별시 동작구 신대방동,344,179,뉴홍현,84.46,201808,30,"42,000",2,2002,보라매로7가길 9,37.493709,126.925927
141924,서울특별시 동작구 신대방동,344,179,뉴홍현,84.46,201904,12,"41,800",6,2002,보라매로7가길 9,37.493709,126.925927


In [703]:
pred.query("index in [57983,57984,57985,57986,141923,141924]")

,exclusive_area,floor,apt_lat,apt_lng,code,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price),transaction_y,transaction_real_price
57983,84.46,2,37.493668,126.925918,1159010900,17,52.0,8,1374,234.0,34,2.181132,0.702318,0.376054,0.336717,68673.0,84016.0,73303.0,2017,58042.0
57984,84.46,4,37.493668,126.925918,1159010900,17,52.0,8,1374,234.0,34,2.181132,0.702318,0.376054,0.336717,68673.0,84016.0,73303.0,2017,59334.0
57985,84.46,4,37.493668,126.925918,1159010900,17,52.0,8,1374,234.0,34,2.181132,0.702318,0.376054,0.336717,68673.0,84016.0,73303.0,2017,59334.0
57986,84.46,5,37.493668,126.925918,1159010900,17,52.0,8,1374,234.0,34,2.181132,0.702318,0.376054,0.336717,68673.0,84016.0,73303.0,2018,59789.0
141923,84.46,2,37.493709,126.925927,1159010900,17,52.0,8,1374,234.0,34,2.176642,0.697816,0.373198,0.340548,68673.0,84016.0,73303.0,2018,44787.0
141924,84.46,6,37.493709,126.925927,1159010900,17,52.0,8,1374,234.0,34,2.176642,0.697816,0.373198,0.340548,68673.0,84016.0,73303.0,2019,46848.0


### 오차율 최소값 데이터

In [667]:
error_rat[error_rat == 0]  # 오차율 0으로 정확하게 예측한 데이터의 인덱스번호

28326     0.0
31482     0.0
90550     0.0
93588     0.0
97671     0.0
123260    0.0
126252    0.0
188619    0.0
238735    0.0
Name: transaction_real_price, dtype: float64

In [600]:
pred.query("index in [28326,31482,90550,93588,97671,123260,126252,188619,238735]")

,exclusive_area,floor,apt_lat,apt_lng,code,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price),transaction_y,transaction_real_price
28326,15.630,4,37.547052,127.071819,1121510700,10,38.0,14,1290,197.0,37,1.834207,0.272184,0.610588,0.648281,74841.0,64696.0,62584.0,2017,15600.0
31482,84.980,8,37.489737,126.893869,1153010200,13,47.0,12,1281,334.0,38,3.868626,0.397854,0.215771,0.471713,74841.0,84016.0,73303.0,2017,44500.0
90550,84.950,5,37.533645,127.115145,1171010300,19,159.0,28,3153,409.0,53,0.922956,0.598829,0.275774,1.988576,68673.0,52210.0,73303.0,2017,48500.0
93588,79.600,6,37.518144,126.840849,1147010300,6,96.0,11,3186,316.0,50,5.151623,1.085349,0.093170,1.477129,50860.0,52210.0,72375.0,2018,33500.0
97671,120.038,23,37.510321,126.900177,1156011800,18,42.0,30,1401,266.0,41,2.314846,0.803268,0.211062,0.834204,68673.0,64696.0,73303.0,2017,74800.0
123260,38.920,4,37.489044,126.954613,1162010100,4,76.0,11,1350,271.0,36,2.308084,0.779836,0.636587,0.851646,68673.0,64696.0,62584.0,2018,19800.0
126252,64.390,13,37.502485,126.860511,1153010600,13,47.0,12,1281,334.0,38,5.157255,0.892177,0.306241,1.547998,68673.0,52210.0,73303.0,2018,33600.0
188619,47.000,5,37.504975,126.866779,1153010600,13,47.0,12,1281,334.0,38,4.570981,0.829753,0.049299,1.514991,68673.0,52210.0,72375.0,2019,18850.0
238735,155.586,14,37.524830,126.870023,1147010200,6,96.0,11,3186,316.0,50,3.060743,0.456430,0.395599,0.188744,74841.0,93476.0,73303.0,2019,138000.0


In [602]:
accurate = pd.read_csv('data/ProcessedData/test_final.csv', encoding='cp949')
accurate.query("index in [28326,31482,90550,93588,97671,123260,126252,188619,238735]")

,시군구,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,위도,경도
28326,서울특별시 광진구 화양동,113,1,광진코지웰,15.630,201709,26,"15,600",4,2014,능동로19길 35,37.547052,127.071819
31482,서울특별시 구로구 구로동,1260,0,이화우성,84.980,201707,31,"44,500",8,1998,도림로22길 8,37.489737,126.893869
90550,서울특별시 송파구 풍납동,224,1,케이앤피상상,84.950,201709,13,"48,500",5,2008,풍성로6가길 7,37.533645,127.115145
93588,서울특별시 양천구 신월동,545,2,흥인아이젠,79.600,201803,24,"33,500",6,2011,신월로 155,37.518144,126.840849
97671,서울특별시 영등포구 도림동,821,0,영등포아트자이,120.038,201705,17,"74,800",23,2014,도영로 66,37.510321,126.900177
123260,서울특별시 관악구 봉천동,1021,0,일두(1021),38.920,201809,29,"19,800",4,1985,은천로39길 56,37.489044,126.954613
126252,서울특별시 구로구 고척동,141,1,서울가든,64.390,201808,21,"33,600",13,1989,중앙로5길 62,37.502485,126.860511
188619,서울특별시 구로구 고척동,57,9,산업인,47.000,201910,8,"18,850",5,1976,경인로47다길 43,37.504975,126.866779
238735,서울특별시 양천구 목동,961,0,하이페리온2,155.586,201909,18,"138,000",14,2006,오목로 300,37.524830,126.870023


In [699]:
feature.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 803001 entries, 0 to 803000
Data columns (total 19 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   exclusive_area           803001 non-null  float64
 1   floor                    803001 non-null  int64  
 2   apt_lat                  803001 non-null  float64
 3   apt_lng                  803001 non-null  float64
 4   code                     803001 non-null  int64  
 5   subway_cnt               803001 non-null  int64  
 6   park_cnt                 803001 non-null  float64
 7   starbucks_cnt            803001 non-null  int64  
 8   academy_cnt              803001 non-null  int64  
 9   daycare_cnt              803001 non-null  float64
 10  kindergarden_cnt         803001 non-null  int64  
 11  dist_river               803001 non-null  float64
 12  dist_subway              803001 non-null  float64
 13  dist_park                803001 non-null  float64
 14  dist

### 종합 평가

In [726]:
max_rat = [249.023529,301.333333,298.773346,259.093333]
np.mean(max_rat)

277.05588525

In [733]:
73805/254674*100

28.980186434422045

<평가차수 : 1회차 / 평가일 : 20.06.06 / 학습&예측 데이터 열 개수 19개>

<파생변수 : 아파트 위도&경도, 법정동코드, 구별 지하철&공원&학원&어린이집&유치원&스타벅스 빈도수,
    한강&지하철&공원&스타벅스까지의 최소거리(각 아파트마다), 스(타벅스)세권&역세권&숲세권, 거래년도 / 17개>

<사용 알고리즘>
1)         LightGBM Regressor : 평균 오차율 22.28% / 1분위수 10.17% / 유효 예측값 개수 87578개, 34.38%
2)              XGB Regressor : 평균 오차율 22.74% / 1분위수 10.56% / 유효 예측값 개수 89653개, 35.20%
3) GradientBoosting Regressor : 평균 오차율 21.75% / 1분위수 10.02% / 유효 예측값 개수 81782개, 32.11%
4)         CatBoost Regressor : 평균 오차율 21.99% / 1분위수 10.97% / 유효 예측값 개수 73805개, 28.98%

*부가 설명 : 각 알고리즘마다 평균적으로 (-)277% 정도의 오차율을 가진 값들이 1개씩 존재하는데, 이를 분석해본 결과
    1.오차율 최대값을 가지는 아파트와 동일한 아파트들의 실제 거래금액이 낮음
    2.알고리즘이 해당 아파트들의 가격을 전체적으로 높게 예측함
    3.오차율 최대값을 가지는 아파트의 실제 거래금액이 동일한 이름을 가진 다른 아파트들의 거래금액보다 확연히 낮음

실제값 - 예측값 = 오차값

오차값(절대값).mean() = 평균오차값

( 실제값 - 예측값 ) / 실제값 * 100 = 오차율[ (+) : 실제값 > 예측값, (-) : 실제값 < 예측값 ]

오차율(절대값).mean() = 평균오차율

% 100 - 평균오차율 = 평균예측률